# 04-4 Agent Implementation

- LangGraph + Pydantic v2（仕様準拠、既存ロジック変更なし）
- 入力: `artifacts/{RUN_ID}/handoff/04-3_llm_tools_setup.pkl`
- 出力: `artifacts/{RUN_ID}/handoff/04-4_agent_implementation.pkl`


In [1]:
# === Cell 2: Dynamic RUN_ID Bootstrap (aligned with 04-3) ===
import os, re
from pathlib import Path
from datetime import datetime

def bootstrap_run_id(force_new: bool=False) -> str:
    ROOT = Path(".").resolve()
    ARTIFACTS = ROOT / "artifacts"
    ARTIFACTS.mkdir(parents=True, exist_ok=True)
    env_id = os.environ.get("RUN_ID", "").strip()
    if env_id and not force_new:
        run_id = env_id
    else:
        latest_file = ARTIFACTS / "LATEST_RUN_ID.txt"
        if latest_file.exists() and not force_new:
            run_id = latest_file.read_text().strip()
        else:
            pat = re.compile(r"^\d{4}-\d{2}-\d{2}_\d{6}$")
            dirs = [p.name for p in ARTIFACTS.iterdir() if p.is_dir() and pat.match(p.name)]
            run_id = sorted(dirs, reverse=True)[0] if (dirs and not force_new) else datetime.now().strftime("%Y-%m-%d_%H%M%S")
    (ARTIFACTS / "LATEST_RUN_ID.txt").write_text(run_id)
    os.environ["RUN_ID"] = run_id
    return run_id

RUN_ID = bootstrap_run_id(False)

ARTIFACTS   = Path("artifacts") / RUN_ID
HANDOFF_DIR = ARTIFACTS / "handoff"
RESULTS_DIR = ARTIFACTS / "results"
HANDOFF_DIR.mkdir(parents=True, exist_ok=True)
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

IN_HANDOFF_PKL  = HANDOFF_DIR / "04-3_llm_tools_setup_with_tools.pkl"
OUT_HANDOFF_PKL = HANDOFF_DIR / "04-4_agent_implementation.pkl"

print(f"[RUN_ID] {RUN_ID}")
print(f"[IN]  {IN_HANDOFF_PKL}")
print(f"[OUT] {OUT_HANDOFF_PKL}")


[RUN_ID] 2025-10-22_074213
[IN]  artifacts/2025-10-22_074213/handoff/04-3_llm_tools_setup_with_tools.pkl
[OUT] artifacts/2025-10-22_074213/handoff/04-4_agent_implementation.pkl


In [2]:
# === Cell 2a: Preflight handoff presence ===
from pathlib import Path

if not Path(IN_HANDOFF_PKL).is_file():
    raise FileNotFoundError(f"[04-4] handoff file not found: {IN_HANDOFF_PKL}\n"
                            "→ 04-3 を実行して 04-3_llm_tools_setup.pkl を生成してください。")
print(f"[04-4] handoff file exists: {IN_HANDOFF_PKL}")


[04-4] handoff file exists: artifacts/2025-10-22_074213/handoff/04-3_llm_tools_setup_with_tools.pkl


In [3]:
# === Cell 3: Load handoff payload (STRICT) ===
import pickle

with open(IN_HANDOFF_PKL, "rb") as f:
    payload = pickle.load(f) or {}

required = ["cfg", "tools", "llm"]
missing  = [k for k in required if k not in payload or payload[k] is None]
print("[04-4] handoff keys:", {k: ("OK" if k not in missing else "MISSING") for k in required})

if missing:
    raise RuntimeError(f"[04-4] handoff missing required keys: {missing}"
                       f"@ {IN_HANDOFF_PKL}"
                       "→ 04-3 を修正／再実行して必要キーを含めてください。")

# Bind globals (no logic change)
cfg   = payload["cfg"]
tools = payload["tools"]
llm   = payload["llm"]

# Read-only report of extra/stat keys if present
stats_like = {k: v for k, v in payload.items() if "stat" in k.lower() or "brand" in k.lower() or "cert" in k.lower()}
print("[04-4] extra/stat keys detected:", list(stats_like.keys())[:10])


[04-4] handoff keys: {'cfg': 'OK', 'tools': 'OK', 'llm': 'OK'}
[04-4] extra/stat keys detected: ['brand_keywords', 'cert_full_info_map']


In [4]:
# === Cell 5.5: Structured Output Schema Definition (Pydantic v2) ===
# Purpose: Define Pydantic models to structure ALL agent I/O (LLM outputs & tool returns)
#          with type-safety and rich validation. This cell is intentionally standalone.
#
# Notes:
# - Field descriptions are in English for better LLM alignment; Japanese comments are for maintainers.
# - Backward compatibility: each model exposes `.to_dict()` (alias of `model_dump()`).
# - Pydantic v2 API (`field_validator`, `model_validator`) only; v1 is not required in this repo/runtime.
#
# Ref: https://docs.pydantic.dev/latest/ (v2)
from __future__ import annotations

from typing import Optional, List, Dict, Any, Literal
from datetime import datetime

from pydantic import BaseModel, Field, field_validator, model_validator, ValidationInfo

# -------- Shared literals / limits --------
RiskLevelLiteral = Literal["low", "medium", "high", "critical"]
ActionLiteral    = Literal["block", "warn", "allow", "review"]
StepLiteral      = Literal["initial", "brand_check", "cert_check", "domain_check", "decision", "complete"]

_MAX_REASON_LEN = 4096
_MAX_STR_LEN    = 256
_MAX_LIST_EVIDENCE = 32
_MAX_LIST_PATTERNS = 64

def _coerce_float_01(x: Any, default: float = 0.0) -> float:
    """Coerce to float in [0,1]. If out of range or invalid, clamp to [0,1]."""
    try:
        f = float(x)
    except Exception:
        return default
    if f < 0.0:
        return 0.0
    if f > 1.0:
        return 1.0
    return f

class _ModelMixin:
    """Backward-compat mixin to expose dict-like outputs expected by legacy code."""
    def to_dict(self) -> Dict[str, Any]:
        return self.model_dump()

    @classmethod
    def from_dict(cls, data: Dict[str, Any]):
        return cls(**data)

# -------- 1) BrandAnalysisResult --------
class BrandAnalysisResult(_ModelMixin, BaseModel):
    """Structured result of brand impersonation analysis."""
    is_impersonation: bool = Field(..., description="True if the domain likely impersonates a known brand.")
    detected_brand: Optional[str] = Field(
        None, description="Canonical name of the impersonated brand if detected (normalized), else null.",
        max_length=_MAX_STR_LEN
    )
    confidence: float = Field(0.0, description="Confidence score in [0.0, 1.0] for the impersonation judgment.")
    evidence: List[str] = Field(default_factory=list, description="List of short evidence snippets supporting the judgment.")
    risk_level: RiskLevelLiteral = Field("low", description="Risk level based on brand context and patterns.")

    # Validators
    @field_validator("confidence", mode="before")
    @classmethod
    def _v_confidence(cls, v):
        return _coerce_float_01(v, default=0.0)

    @field_validator("evidence", mode="before")
    @classmethod
    def _v_evidence_len(cls, v):
        v = v or []
        if isinstance(v, (list, tuple)):
            v = list(v)[:_MAX_LIST_EVIDENCE]
        else:
            v = [str(v)][:1]
        return [str(x)[:_MAX_STR_LEN] for x in v]

# -------- 2) CertificateAnalysisResult --------
class CertificateAnalysisResult(_ModelMixin, BaseModel):
    """Security-focused summary of the target's TLS certificate properties."""
    has_certificate: bool = Field(..., description="True if any valid certificate was found for the domain.")
    is_valid: bool = Field(False, description="True if the certificate is currently valid (date range, parsing succeeded).")
    age_days: int = Field(-1, description="Certificate age in days (from NotBefore to now); -1 if unknown/unavailable.")
    issuer: Optional[str] = Field(None, description="Certificate issuer organization (normalized), if available.", max_length=_MAX_STR_LEN)
    is_free_ca: bool = Field(False, description="True if a free CA (e.g., Let's Encrypt) issued the certificate.")
    is_wildcard: bool = Field(False, description="True if a wildcard certificate (*.example.com).")
    san_count: int = Field(0, description="Number of SAN entries on the certificate.")
    risk_factors: List[str] = Field(default_factory=list, description="List of risk factors derived from certificate fields.")
    confidence: float = Field(0.0, description="Confidence score in [0.0, 1.0] for the certificate-related assessment.")

    @field_validator("san_count", "age_days", mode="before")
    @classmethod
    def _v_non_negative_or_minus1(cls, v, info: ValidationInfo):
        # san_count: >=0 ; age_days: -1 or >=0
        try:
            iv = int(v)
        except Exception:
            iv = -1 if info.field_name == "age_days" else 0
        if info.field_name == "age_days":
            return iv if iv >= -1 else -1
        return max(0, iv)

    @field_validator("confidence", mode="before")
    @classmethod
    def _v_confidence(cls, v):
        return _coerce_float_01(v, default=0.0)

    @field_validator("risk_factors", mode="before")
    @classmethod
    def _v_risk_factors(cls, v):
        v = v or []
        if isinstance(v, (list, tuple)):
            v = list(v)[:_MAX_LIST_EVIDENCE]
        else:
            v = [str(v)][:1]
        return [str(x)[:_MAX_STR_LEN] for x in v]

    @model_validator(mode="after")
    def _v_is_valid_consistency(self):
        # has_certificate=False => is_valid must be False, san_count=0
        if not self.has_certificate:
            self.is_valid = False
            self.san_count = 0
        return self

# -------- 3) DomainAnalysisResult --------
class DomainAnalysisResult(_ModelMixin, BaseModel):
    """Structured summary of domain structure risks and lexical signals."""
    is_suspicious: bool = Field(..., description="True if domain structure suggests phishing or abuse.")
    domain_length: int = Field(0, description="Length of the full domain string (host only, no scheme).")
    has_suspicious_tld: bool = Field(False, description="True if the TLD is in a known dangerous list.")
    tld: str = Field(..., description="Top-level domain (without leading dot), e.g., 'com', 'co.jp'.")
    typosquatting_candidate: Optional[str] = Field(None, description="Closest brand or domain candidate (if any).", max_length=_MAX_STR_LEN)
    levenshtein_distance: Optional[int] = Field(None, description="Edit distance to the candidate when relevant (non-negative).")
    risk_factors: List[str] = Field(default_factory=list, description="List of risk factor codes or short explanations.")
    confidence: float = Field(0.0, description="Confidence score in [0.0, 1.0] for the domain structural assessment.")

    @field_validator("domain_length", mode="before")
    @classmethod
    def _v_len(cls, v):
        try:
            iv = int(v)
        except Exception:
            iv = 0
        return max(0, iv)

    @field_validator("tld", mode="before")
    @classmethod
    def _v_tld(cls, v):
        v = (v or "").strip()
        if v.startswith("."):
            v = v[1:]
        return v[:_MAX_STR_LEN]

    @field_validator("levenshtein_distance", mode="before")
    @classmethod
    def _v_lev(cls, v):
        if v is None:
            return None
        try:
            iv = int(v)
        except Exception:
            iv = 0
        return max(0, iv)

    @field_validator("risk_factors", mode="before")
    @classmethod
    def _v_rf(cls, v):
        v = v or []
        if isinstance(v, (list, tuple)):
            v = list(v)[:_MAX_LIST_EVIDENCE]
        else:
            v = [str(v)][:1]
        return [str(x)[:_MAX_STR_LEN] for x in v]

    @field_validator("confidence", mode="before")
    @classmethod
    def _v_confidence(cls, v):
        return _coerce_float_01(v, default=0.0)

# -------- 4) PhishingDetectionResult (main output) --------
class PhishingDetectionResult(_ModelMixin, BaseModel):
    """Final phishing verdict with integrated signals from ML, certificate, domain & brand checks."""
    domain: str = Field(..., description="Target domain under evaluation (host only).", min_length=1, max_length=_MAX_STR_LEN)
    is_phishing: bool = Field(..., description="True if the final decision is phishing.")
    confidence: float = Field(..., description="Overall confidence in [0.0, 1.0] for the final decision.")
    risk_level: RiskLevelLiteral = Field(..., description="Normalized risk level summarizing overall severity.")
    ml_probability: float = Field(..., description="Raw ML model probability (0.0 to 1.0).")
    brand_analysis: Optional[BrandAnalysisResult] = Field(None, description="Nested result of brand impersonation analysis.")
    certificate_analysis: Optional[CertificateAnalysisResult] = Field(None, description="Nested result of certificate analysis.")
    domain_analysis: Optional[DomainAnalysisResult] = Field(None, description="Nested result of domain structure analysis.")
    reasoning: str = Field(..., description="Concise explanation describing why the decision was made.", max_length=_MAX_REASON_LEN)
    detected_patterns: List[str] = Field(default_factory=list, description="Patterns detected during analysis (codes or short strings).")
    recommended_action: ActionLiteral = Field("review", description="Recommended action for downstream systems or analysts.")

    @field_validator("confidence", "ml_probability", mode="before")
    @classmethod
    def _v_range_01(cls, v):
        return _coerce_float_01(v, default=0.0)

    @field_validator("detected_patterns", mode="before")
    @classmethod
    def _v_patterns(cls, v):
        v = v or []
        if isinstance(v, (list, tuple)):
            v = list(v)[:_MAX_LIST_PATTERNS]
        else:
            v = [str(v)][:1]
        return [str(x)[:_MAX_STR_LEN] for x in v]

    @model_validator(mode="after")
    def _v_action_coherence(self):
        # If risk is high/critical, never return 'allow'
        if self.risk_level in ("high", "critical") and self.recommended_action == "allow":
            self.recommended_action = "warn"
        return self

# -------- 5) AgentState (LangGraph state) --------
class AgentState(_ModelMixin, BaseModel):
    """Internal state for LangGraph-based agent orchestration."""
    current_step: StepLiteral = Field("initial", description="Current step in the agent pipeline.")
    domain: Optional[str] = Field(None, description="Domain being processed, if any.", max_length=_MAX_STR_LEN)
    ml_probability: Optional[float] = Field(None, description="ML probability available at state time; [0.0, 1.0].")
    intermediate_results: Dict[str, Any] = Field(default_factory=dict, description="Arbitrary map to store tool and node results.")
    final_result: Optional[PhishingDetectionResult] = Field(None, description="Finalized detection result if complete.")
    error: Optional[str] = Field(None, description="Latest error message, if any.", max_length=1024)
    retry_count: int = Field(0, description="How many times the agent retried recoverable failures.")

    @field_validator("retry_count", mode="before")
    @classmethod
    def _v_retry_non_negative(cls, v):
        try:
            iv = int(v)
        except Exception:
            iv = 0
        return max(0, iv)

    @field_validator("ml_probability", mode="before")
    @classmethod
    def _v_mlprob_range(cls, v):
        if v is None:
            return None
        return _coerce_float_01(v, default=0.0)

    @model_validator(mode="after")
    def _v_consistency(self):
        # When complete, final_result should exist; if not, write error
        if self.current_step == "complete" and self.final_result is None:
            self.error = self.error or "State is 'complete' but final_result is None."
        return self

# ---- Public exports ----
__all__ = [
    "BrandAnalysisResult",
    "CertificateAnalysisResult",
    "DomainAnalysisResult",
    "PhishingDetectionResult",
    "AgentState",
    "RiskLevelLiteral",
    "ActionLiteral",
    "StepLiteral",
]

# -------------------------
# Usage examples (commented)
# -------------------------
# result = PhishingDetectionResult(
#     domain="example.com",
#     is_phishing=True,
#     confidence=0.85,
#     risk_level="high",
#     ml_probability=0.30,
#     reasoning="High similarity to brand login + dangerous TLD + very new cert."
# )
#
# # LLM structured output (LangChain-style):
# #   llm_with_structured_output = llm.with_structured_output(PhishingDetectionResult)
# #   response = llm_with_structured_output.invoke("Please analyze this domain: example.com")
# #
# # Backward compatibility:
# #   payload = result.to_dict()


In [5]:
# === セル6（LangGraph-based Phishing Detection Agent）===
# 目的: 既存の PhishingDetectionAgent / evaluate_domain を定義し、LangGraph でのワークフローを提供。
# 互換: 既存 evaluate_domain を維持。_call_tool / _structured / _to_dict を提供。
# 失敗時: langgraph 等の import に失敗した場合は、直列実行のフォールバックで evaluate() を提供。

from __future__ import annotations
import json, time, traceback, builtins
from typing import Any, Dict, Optional

print(">> [Cell6] bootstrapping LangGraph-based agent…")

# --- Helpers & model adapters -------------------------------------------------

def _now_ms() -> int:
    import time as _t
    return int(_t.time() * 1000)

def _to_dict(obj: Any) -> Dict[str, Any]:
    if isinstance(obj, dict):
        return obj
    for attr in ("model_dump", "dict"):
        if hasattr(obj, attr):
            try:
                return getattr(obj, attr)()
            except Exception:
                pass
    try:
        return json.loads(getattr(obj, "json")())
    except Exception:
        return {"value": repr(obj)}

def _structured(kind: str, data: Dict[str, Any]) -> Dict[str, Any]:
    Model = globals().get(kind, None)
    if Model is None:
        return dict(data)
    try:
        if hasattr(Model, "__call__"):
            inst = Model(**data)
            if hasattr(inst, "model_dump"):
                try:
                    return inst.model_dump()
                except Exception:
                    pass
            if hasattr(inst, "__dict__"):
                return dict(inst.__dict__)
        return dict(data)
    except Exception:
        return dict(data)

def _risk_to_score(level: str) -> int:
    m = {"low": 20, "medium": 50, "medium-high": 70, "high": 85, "critical": 95}
    return m.get(str(level).lower(), 0)

def _ensure_cfg(base: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    cfg0: Dict[str, Any] = {}
    if isinstance(base, dict):
        cfg0.update(base)
    g = globals()
    if isinstance(g.get("cfg"), dict):
        tmp = g["cfg"].copy()
        tmp.update(cfg0)
        cfg0 = tmp
    try:
        from _compat import config as _cfgm
        if hasattr(_cfgm, "load_configuration"):
            c2 = _cfgm.load_configuration(cfg_override=cfg0)
            if isinstance(c2, dict):
                return c2
    except Exception:
        pass
    return cfg0

# Provide a proper global `_call_tool` used by wrappers & graph nodes
def _call_tool(tool_name: str, **kwargs) -> Dict[str, Any]:
    g = globals()
    fn = g.get(tool_name)
    if fn is None:
        return {"success": False, "error": f"Tool `{tool_name}` not available"}
    try:
        res = fn(**kwargs)
        out = _to_dict(res)
        if isinstance(out, dict) and "success" not in out:
            out["success"] = True
        return out
    except Exception as e:
        return {"success": False, "error": f"{tool_name} failed: {type(e).__name__}: {e}", "traceback": traceback.format_exc()}

globals()['_call_tool'] = _call_tool

# --- Optional LangGraph import ------------------------------------------------
HAVE_LG = True
try:
    from lib.langgraph import graph as _lg  # project vendored alias if present
    StateGraph, END, START = _lg.StateGraph, _lg.END, _lg.START
except Exception:
    try:
        from langgraph import StateGraph, END, START
    except Exception as e:
        HAVE_LG = False
        _LG_IMPORT_ERROR = e

# --- Agent definition ---------------------------------------------------------

class PhishingDetectionAgent:
    """LangGraph-based (with graceful fallback) phishing detection orchestrator.
    Exposes:
      - evaluate(domain, ml_probability, trace=None) -> PhishingDetectionResult (dict)
      - batch_evaluate(domains, probs) -> List[PhishingDetectionResult]
    """

    def __init__(self, cfg: Optional[Dict[str, Any]] = None):
        self.cfg: Dict[str, Any] = _ensure_cfg(cfg)
        self.graph = None
        if HAVE_LG:
            try:
                self.graph = self._build_graph()
            except Exception:
                print("!! [Cell6] LangGraph build failed, falling back to sequential:", _LG_IMPORT_ERROR)
                self.graph = None
        else:
            print("!! [Cell6] langgraph not available → using sequential fallback")

    def _build_graph(self):
        def brand_node(state: Dict[str, Any]) -> Dict[str, Any]:
            domain = state.get("domain"); mlp = float(state.get("ml_probability") or 0.0)
            try:
                tool = "brand_impersonation_check"
                if "wordlist_detector" in globals():
                    tool = "wordlist_detector"
                out = _call_tool(tool, domain=domain, ml_probability=mlp, cfg=self._ctx())
                res = {
                    "is_impersonation": bool(out.get("is_impersonation") or out.get("is_suspicious")),
                    "confidence": float(out.get("confidence", 0.0) or 0.0),
                    "risk_level": out.get("risk_level", "low"),
                    "detected_brands": out.get("detected_brands") or out.get("brands") or [],
                    "risk_factors": out.get("risk_factors", {}),
                    "reasons": out.get("reasons", []),
                    "success": out.get("success", True),
                    "error": out.get("error"),
                }
                st = dict(state)
                st.setdefault("intermediate_results", {})
                st["intermediate_results"]["brand"] = _structured("BrandAnalysisResult", res)
                st["current_step"] = "certificate_check" if (res["is_impersonation"] or _risk_to_score(res["risk_level"])>=70) else "domain_check"
                return st
            except Exception as e:
                st = dict(state); st.setdefault("intermediate_results", {})
                st["intermediate_results"]["brand"] = _structured("BrandAnalysisResult", {
                    "is_impersonation": False, "confidence": 0.0, "risk_level":"low",
                    "risk_factors": {"error": str(e)}, "reasons": ["brand node error"], "success": False
                })
                st["current_step"] = "certificate_check"
                return st

        def cert_node(state: Dict[str, Any]) -> Dict[str, Any]:
            domain = state.get("domain"); mlp = float(state.get("ml_probability") or 0.0)
            out = _call_tool("certificate_analysis", domain=domain, ml_probability=mlp, cfg=self._ctx())
            res = {
                "confidence": float(out.get("confidence", 0.0) or 0.0),
                "risk_level": out.get("risk_level", "low"),
                "risk_factors": out.get("risk_factors", {}),
                "reasons": out.get("reasons", []),
                "success": out.get("success", True),
                "error": out.get("error"),
            }
            st = dict(state); st.setdefault("intermediate_results", {})
            st["intermediate_results"]["cert"] = _structured("CertificateAnalysisResult", res)
            st["current_state"] = "domain_check"
            return st

        def domain_node(state: Dict[str, Any]) -> Dict[str, Any]:
            domain = state.get("domain"); mlp = float(state.get("ml_probability") or 0.0)
            out = _call_tool("short_domain_analysis", domain=domain, ml_probability=mlp, cfg=self._ctx())
            res = {
                "is_suspicious": bool(out.get("is_suspicious", False)),
                "confidence": float(out.get("confidence", 0.0) or 0.0),
                "risk_level": out.get("risk_level", "low"),
                "risk_factors": out.get("risk_factors", {}),
                "reasons": out.get("reasons", []),
                "success": out.get("success", True),
                "error": out.get("error"),
            }
            st = dict(state); st.setdefault("intermediate_results", {})
            st["intermediate_results"]["domain"] = _structured("DomainAnalysisResult", res)
            st["current_state"] = "decision"
            return st

        def decide_node(state: Dict[str, Any]) -> Dict[str, Any]:
            brand = state.get("intermediate_results", {}).get("brand", {}) or {}
            cert  = state.get("intermediate_results", {}).get("cert", {}) or {}
            dom   = state.get("intermediate_results", {}).get("domain", {}) or {}
            mlp   = float(state.get("ml_probability") or 0.0)

            conf = max(float(brand.get("confidence",0.0) or 0.0),
                       float(dom.get("confidence",0.0) or 0.0),
                       float(cert.get("confidence",0.0) or 0.0))
            risk_score = max(
                int(conf * 100),
                _risk_to_score(brand.get("risk_level")),
                _risk_to_score(dom.get("risk_level")),
                _risk_to_score(cert.get("risk_level"))
            )
            is_phish = bool(brand.get("is_impersonation")) or bool(dom.get("is_suspicious")) or (risk_score >= 50)

            lvl = "low"
            if   risk_score >= 85: lvl = "critical"
            elif risk_score >= 70: lvl = "high"
            elif risk_score >= 50: lvl = "medium-high"
            elif risk_score >= 30: lvl = "medium"

            result = {
                "domain": state.get("name") or state.get("domain"),
                "ml_probability": mlp,
                "is_phishing": bool(is_phish),
                "confidence": float(max(0.0, min(1.0, conf))),
                "risk_level": lvl,
                "detected_brands": brand.get("lures", []) or brand.get("detected_brands", []),
                "risk_factors": {
                    "brand":       brand.get("risk_factors", {}),
                    "domain":      dom.get("risk_factors", {}),
                    "certificate": cert.get("risk_factors", {}),
                },
                "reasons": (brand.get("reasons", []) or []) + (dom.get("reasons", []) or []) + (cert.get("reasons", []) or []),
                "success": bool(brand.get("success", True) and dom.get("success", True) and cert.get("success", True)),
            }
            st = dict(state); st["result"] = _structured("PhishingDetectionResult", result); st["current_state"] = "done"
            return st

        g = StateGraph(dict)
        g.add_node("brand_check",       lambda s: brand_node(s))
        g.add_node("certificate_check", lambda s: cert_node(s))
        g.add_node("domain_check",      lambda s: domain_node(s))
        g.add_node("decision",          lambda s: decide_node(s))
        g.set_entry_point("brand_check")
        g.add_conditional_edges("brand_check", lambda s: s.get("current_state","domain_check"),
                                {"certificate_check":"certificate_check","domain_check":"domain_check"})
        g.add_edge("certificate_check", "domain_check")
        g.add_conditional_edges("domain_check", lambda s: "decision", {"decision":"decision"})
        g.add_edge("decision", END)
        return g

    def _ctx(self) -> Dict[str, Any]:
        ctx = {"cfg": self.cfg}
        return ctx

    def evaluate(self, domain: str, ml_probability: float, trace: Optional[Any] = None) -> Dict[str, Any]:
        state0 = {
            "name": str(domain),
            "domain": str(domain),
            "ml_probability": float(ml_probability or 0.0),
            "intermediate_results": {},
            "current_state": "brand_check",
        }
        if self.graph is not None:
            try:
                runtime = self.graph.compile()
                out = runtime.invoke(state0)
                res = out.get("result")
                if res:
                    return _to_dict(res)
            except Exception as e:
                print("!! [Cell6] graph run failed → fallback to sequential:", e)
                print(traceback.format_exc())

        b = _call_tool("brand_impersonation_check", domain=domain, ml_probability=ml_probability, cfg=self._ctx())
        c = _call_tool("certificate_analysis",      domain=domain, ml_probability=ml_probability, cfg=self._ctx())
        d = _call_tool("short_domain_analysis",     domain=domain, ml_probability=ml_probability, cfg=self._ctx())

        conf = max(float(b.get("confidence",0.0) or 0.0),
                   float(c.get("confidence",0.0) or 0.0),
                   float(d.get("confidence",0.0) or 0.0))
        risk_score = max(
            int(conf*100),
            _risk_to_score(b.get("risk_level")),
            _risk_to_score(c.get("risk_level")),
            _risk_to_score(d.get("risk_level"))
        )
        is_phish = bool(b.get("is_phishing") or b.get("is_impersonation")) or bool(d.get("is_suspicious")) or (risk_score >= 50)

        lvl = "low"
        if   risk_score >= 85: lvl = "critical"
        elif risk_score >= 70: lvl = "high"
        elif risk_score >= 50: lvl = "medium-high"
        elif risk_score >= 30: lvl = "medium"

        result = {
            "domain": str(domain),
            "ml_probability": float(ml_probability or 0.0),
            "is_phishing": bool(is_phish),
            "confidence": float(conf),
            "risk_level": lvl,
            "detected_brands": b.get("detected_brands", []),
            "risk_factors": {
                "brand":       b.get("risk_factors", {}),
                "domain":      d.get("risk_factors", {}),
                "certificate": c.get("risk_factors", {}),
            },
            "reasons": (b.get("reasons", []) or []) + (d.get("reasons", []) or []) + (c.get("reasons", []) or []),
            "success": bool(b.get("success", True) and c.get("success", True) and d.get("success", True)),
        }
        return _structured("PhishingDetectionResult", result)

    def batch_evaluate(self, domains, probs):
        out = []
        for dd, pp in zip(list(domains), list(probs)):
            try:
                out.append(self.evaluate(dd, pp))
            except Exception as e:
                out.append({"domain": dd, "ml_probability": float(pp or 0.0),
                            "is_phishing": False, "confidence": 0.0, "risk_level":"low",
                            "success": False, "error": str(e)})
        return out

# Backward-compatible alias
def evaluate_domain(domain: str, ml_probability: float, **kwargs) -> Any:
    agent = PhishingDetectionAgent(cfg=globals().get("cfg"))
    return agent.evaluate(domain, ml_probability, **kwargs)

print("✅ LangGraph-based PhishingDetectionAgent defined; evaluate_domain bound.",
      "(graph available)" if HAVE_LG else "(fallback mode: sequential)")


>> [Cell6] bootstrapping LangGraph-based agent…
✅ LangGraph-based PhishingDetectionAgent defined; evaluate_domain bound. (fallback mode: sequential)


In [6]:
# === P1-3b: Local tool wrapper `certificate_analysis` (for 04-4 globals) ===
from typing import Dict, Any, Optional, List, Tuple
from datetime import datetime

_FREE_CA_HINTS = (
"let's encrypt","zerossl","buypass","trustasia","gandi","cloudflare","google internet authority","isrg","actalis")

def _p1_3b_norm_apostrophes(s: str) -> str:
    return (s or '').lower().replace('’', "'").replace('‘', "'").replace('`', "'")

def _p1_3b_risk_level(score: float) -> str:
    if score >= 80: return "critical"
    if score >= 60: return "high"
    if score >= 45: return "medium-high"
    if score >= 25: return "medium"
    return "low"

def _p1_3b_norm_dt(s: Any) -> Optional[datetime]:
    if not s: return None
    if isinstance(s, datetime): return s
    if isinstance(s, (int, float)):
        try: return datetime.utcfromtimestamp(float(s))
        except Exception: return None
    s = str(s).strip().replace("Z", "")
    for fmt in ("%Y-%m-%dT%H:%M:%S", "%Y-%m-%d %H:%M:%S", "%Y-%m-%d"):
        try: return datetime.strptime(s, fmt)
        except Exception: pass
    try: return datetime.fromisoformat(s)
    except Exception: return None

def _p1_3b_days_between(a: Any, b: Any) -> Optional[int]:
    da, db = _p1_3b_norm_dt(a), _p1_3b_norm_dt(b)
    if da and db:
        try: return abs((db - da).days)
        except Exception: return None
    return None

def _p1_3b_pick_cert_record(payload_or_globals: Dict[str, Any], domain: str) -> Dict[str, Any]:
    cfmap = None
    if isinstance(payload_or_globals, dict):
        cfmap = payload_or_globals.get("cert_full_info_map") or payload_or_globals.get("CERT_FULL_INFO_MAP")
    if not isinstance(cfmap, dict):
        cfmap = globals().get("CERT_FULL_INFO_MAP") or globals().get("cert_full_info_map") or {}
    if not isinstance(cfmap, dict):
        cfmap = {}
    # direct and simple variants
    rec = cfmap.get(domain) or cfmap.get(domain.lower())
    if not isinstance(rec, dict):
        # try www/no-www
        if domain.startswith("www."):
            rec = cfmap.get(domain[4:], {})
        else:
            rec = cfmap.get("www."+domain, {})
    if not isinstance(rec, dict):
        # try eTLD+1 fallback (naive split)
        parts = domain.split(".")
        if len(parts) >= 2:
            rec = cfmap.get(".".join(parts[-2:]), {})
    return rec if isinstance(rec, dict) else {}

def certificate_analysis(domain: str, ml_probability: float = 0.0, cfg: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    d = (domain or "").strip().lower()
    try:
        p = float(ml_probability or 0.0)
    except Exception:
        p = 0.0

    payload = globals().get("payload") or {}
    rec = _p1_3b_pick_cert_record(payload, d)
    if (not isinstance(rec, dict) or not rec) and isinstance(cfg, dict):
        if cfg.get('cert_full_info_map') or cfg.get('CERT_FULL_INFO_MAP'):
            rec = _p1_3b_pick_cert_record({'cert_full_info_map': cfg.get('cert_full_info_map') or cfg.get('CERT_FULL_INFO_MAP')}, d)

    issuer = (str(rec.get("issuer") or rec.get("issuer_cn") or rec.get("ca") or "")).strip()
    org    = (str(rec.get("organization") or rec.get("org") or "")).strip()
    valid_days = rec.get("valid_days")
    if valid_days is None:
        valid_days = _p1_3b_days_between(rec.get("valid_from") or rec.get("not_before"),
                                         rec.get("valid_to")   or rec.get("not_after"))
    try:
        valid_days = int(valid_days) if valid_days is not None else None
    except Exception:
        valid_days = None

    san_count = rec.get("san_count")
    if san_count is None:
        sans = rec.get("subject_alt_names") or rec.get("dns_names") or rec.get("san") or []
        try: san_count = len(sans)
        except Exception: san_count = None

    self_signed = bool(rec.get("self_signed") or rec.get("is_self_signed", False))
    wildcard    = bool(rec.get("wildcard") or rec.get("is_wildcard", False))

    issuer_lc = issuer.lower()
    free_ca = any(h in issuer_lc for h in _FREE_CA_HINTS) if issuer else False
    very_new = (valid_days is not None and valid_days <= 120)

    score = 0.0
    if self_signed: score += 35
    if not org:     score += 15
    if free_ca:     score += 10
    if very_new:    score += 10
    if wildcard:    score += 5
    if isinstance(san_count, int) and san_count <= 1: score += 5
    score += 8.0 * float(p or 0.0)
    score = max(0.0, min(score, 99.0))

    level = _p1_3b_risk_level(score)
    conf  = round(min(1.0, max(0.0, (score / 100.0) + 0.10 * float(p or 0.0))), 3)

    reasons: List[str] = []
    if self_signed: reasons.append("self-signed certificate")
    if not org:     reasons.append("organization field missing (DV)")
    if free_ca:     reasons.append("issued by widely-used free CA")
    if very_new:    reasons.append(f"short validity ({valid_days} days)" if valid_days is not None else "short validity period")
    if wildcard:    reasons.append("wildcard certificate detected")
    if isinstance(san_count, int) and san_count <= 1: reasons.append("SAN count is small")
    if not reasons:
        reasons.append("no strong anomalies found in certificate metadata")

    # Provide both risk_factors.* and top-level compatibility flags (used by 04-4 node)
    result = {
        "confidence": conf,
        "risk_level": level,
        "risk_factors": {
            "issuer": issuer or None,
            "organization_present": bool(org),
            "valid_days": valid_days,
            "san_count": san_count,
            "self_signed": self_signed,
            "wildcard": wildcard,
            "free_ca": free_ca,
            "very_new_cert": very_new,
            "ml_probability": p,
        },
        "is_free_ca": free_ca,
        "very_new": very_new,
        "is_wildcard": wildcard,
        "reasons": reasons,
        "success": True,
    }
    return result

print("✅ Local tool defined: certificate_analysis (P1-3b)")


✅ Local tool defined: certificate_analysis (P1-3b)


In [7]:
# === P1-3b TEST (local, no external I/O) ===
if "payload" not in globals() or not isinstance(payload, dict):
    payload = {}
# Unconditionally set a minimal cert record for the test domain
payload["cert_full_info_map"] = {
    "paypal-secure-login.info": {
        "issuer": "Let's Encrypt",
        "organization": "",
        "valid_days": 90,
        "san_count": 1,
        "self_signed": False,
        "wildcard": False,
    }
}
_t = certificate_analysis("paypal-secure-login.info", 0.34, cfg=globals().get("cfg"))
print("P1-3b test result:", _t)
assert isinstance(_t, dict) and "confidence" in _t and "risk_level" in _t and "risk_factors" in _t and "success" in _t
assert _t["risk_factors"].get("free_ca") is True
print("P1-3b smoketest: OK")


P1-3b test result: {'confidence': 0.461, 'risk_level': 'medium', 'risk_factors': {'issuer': "Let's Encrypt", 'organization_present': False, 'valid_days': 90, 'san_count': 1, 'self_signed': False, 'wildcard': False, 'free_ca': True, 'very_new_cert': True, 'ml_probability': 0.34}, 'is_free_ca': True, 'very_new': True, 'is_wildcard': False, 'reasons': ['organization field missing (DV)', 'issued by widely-used free CA', 'short validity (90 days)', 'SAN count is small'], 'success': True}
P1-3b smoketest: OK


In [8]:
# === Cell NEW: Restore compatibility wrapper `_execute_tool_with_trace` ===
# Purpose: Provide a thin wrapper around `_call_tool` that records per-tool traces.
# Rationale: The original 03_ai_agent_analysis.ipynb exposed `_execute_tool_with_trace(self, tool_name, inputs, trace)`
# as a method on the agent. The refactored split notebooks switched to a module-level `_call_tool`.
# This wrapper preserves the legacy semantics (return shape + optional tracing) with minimal surface change.
from typing import Any, Dict, Optional
import time

def _execute_tool_with_trace(tool_name: str, inputs: Dict[str, Any], trace: Optional[Any] = None) -> Dict[str, Any]:
    """
    トレース付きツール実行（互換ラッパ）
    - calls: `_call_tool(tool_name, **inputs)`
    - ensures: result shape is {"success": True, "result": {...}} when the tool returns a non-error payload
    - if `trace` has `add_tool_trace(...)`, record execution metadata.
    """
    start = time.time()
    # Ensure dict
    inputs = inputs or {}
    # Execute via the unified tool dispatcher
    out = _call_tool(tool_name, **inputs)
    # Normalize shape to keep legacy-compatible contract
    if not isinstance(out, dict):
        out = {"success": True, "result": {"value": repr(out)}}
    else:
        if "success" not in out:
            out = {"success": True, "result": out}
    # Trace
    if trace is not None and hasattr(trace, "add_tool_trace"):
        try:
            trace.add_tool_trace(tool_name=tool_name, inputs=inputs, output=out, duration=time.time()-start)
        except Exception:
            # Tracing is best-effort; do not disrupt main flow
            pass
    return out

In [9]:

# === P1-2b: Local tool wrapper `brand_impersonation_check` (for 04-4 globals) ===
from typing import Dict, Any, Iterable, List, Tuple, Optional
import re

_P1_2B_RISK_WORDS = {'secure','login','verify','update','account','wallet','support','confirm','bank','invoice','payment'}

def _p1_2b_risk_level(conf: float) -> str:
    if conf >= 0.85: return "critical"
    if conf >= 0.70: return "high"
    if conf >= 0.50: return "medium-high"
    if conf >= 0.30: return "medium"
    return "low"

def _p1_2b_iter_keywords(bk: Any):
    if bk is None:
        return []
    if isinstance(bk, dict):
        for k, v in bk.items():
            if isinstance(k, str) and k: yield k.lower()
            if isinstance(v, (list, tuple, set)):
                for x in v:
                    if isinstance(x, str) and x: yield x.lower()
            elif isinstance(v, str) and v: yield v.lower()
        return
    if isinstance(bk, (list, tuple, set)):
        for x in bk:
            if isinstance(x, str) and x: yield x.lower()
        return
    try:
        for x in list(bk):
            if isinstance(x, str) and x: yield x.lower()
    except Exception:
        return []

def _p1_2b_tokenize_domain(d: str) -> List[str]:
    try:
        d = d.encode('idna').decode('ascii')
    except Exception:
        pass
    return [t for t in re.split(r'[^a-z0-9]+', (d or '').lower()) if t]

def _p1_2b_one_edit(a: str, b: str) -> bool:
    la, lb = len(a), len(b)
    if abs(la - lb) > 1: return False
    if la > lb: a, b, la, lb = b, a, lb, la
    i = j = edits = 0
    while i < la and j < lb:
        if a[i] == b[j]:
            i += 1; j += 1
        else:
            edits += 1
            if edits > 1: return False
            if la == lb: i += 1; j += 1
            else: j += 1
    edits += (lb - j) + (la - i)
    return edits <= 1

def brand_impersonation_check(domain: str, ml_probability: float = 0.0, cfg: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    d = (domain or "").strip().lower()
    try:
        p = float(ml_probability or 0.0)
    except Exception:
        p = 0.0

    g = globals()
    bk = g.get("BRAND_KEYWORDS", g.get("brand_keywords", None))
    if bk is None and "payload" in g and isinstance(g["payload"], dict):
        bk = g["payload"].get("brand_keywords")

    brands = sorted(set(_p1_2b_iter_keywords(bk)))
    tokens = _p1_2b_tokenize_domain(d)

    substr_hits = [b for b in brands if b and b in d]
    token_hits  = [b for b in brands if b in tokens]

    typo_hits: List[Tuple[str, str]] = []
    cand = [t for t in tokens if len(t) >= 3]
    for b in brands:
        if not b or len(b) < 4: 
            continue
        if b in substr_hits or b in token_hits:
            continue
        for t in cand:
            if _p1_2b_one_edit(b, t):
                typo_hits.append((b, t))
                break

    risk_kw = [kw for kw in _P1_2B_RISK_WORDS if kw in d]

    score = 0.0
    if substr_hits or token_hits: score += 0.60
    if typo_hits: score += 0.25
    if risk_kw: score += 0.10 + 0.02 * min(len(risk_kw), 5)
    score += 0.30 * p
    score = max(0.0, min(score, 0.99))

    result = {
        "is_impersonation": bool(substr_hits or token_hits or typo_hits),
        "confidence": score,
        "risk_level": _p1_2b_risk_level(score),
        "detected_brands": sorted(set(substr_hits + token_hits)),
        "risk_factors": {
            "typosquatting_matches": typo_hits,
            "risk_keywords": risk_kw,
            "ml_probability": p,
            "tokens": tokens[:8],
        },
        "reasons": [
            "brand keyword present" if (substr_hits or token_hits) else "no direct brand keyword found",
            "typosquatting candidate detected" if typo_hits else "no typo candidate",
            f"ml_probability contribution={round(0.30*p,3)}"
        ],
        "success": True,
    }
    return result

print("✅ Local tool defined: brand_impersonation_check (P1-2b fixed)")


✅ Local tool defined: brand_impersonation_check (P1-2b fixed)


In [10]:
# === P1-2b TEST (local, no external I/O) ===
try:
    _bk = (globals().get("payload") or {}).get("brand_keywords")
    if not _bk: BRAND_KEYWORDS = ["paypal","mercari","ledger"]
except Exception:
    BRAND_KEYWORDS = ["paypal","mercari","ledger"]
_t = brand_impersonation_check("paypal-secure-login.info", 0.34, cfg=globals().get("cfg"))
print("P1-2b test result:", _t)
assert isinstance(_t, dict) and "confidence" in _t and "risk_level" in _t and "success" in _t
if "paypal" in [k.lower() for k in globals().get("BRAND_KEYWORDS", [])]:
    assert _t.get("is_impersonation") is True
print("P1-2b smoketest: OK")


P1-2b test result: {'is_impersonation': True, 'confidence': 0.842, 'risk_level': 'high', 'detected_brands': ['paypal'], 'risk_factors': {'typosquatting_matches': [], 'risk_keywords': ['login', 'secure'], 'ml_probability': 0.34, 'tokens': ['paypal', 'secure', 'login', 'info']}, 'reasons': ['brand keyword present', 'no typo candidate', 'ml_probability contribution=0.102'], 'success': True}
P1-2b smoketest: OK


In [11]:

# === P1-2b TEST (local tool; no external I/O) ===
try:
    if "BRAND_KEYWORDS" not in globals():
        BRAND_KEYWORDS = ["paypal", "mercari", "ledger"]
    _out = brand_impersonation_check("paypal-secure-login.info", 0.34)
    print("P1-2b test result:", _out)
    assert isinstance(_out, dict) and all(k in _out for k in ["is_impersonation","confidence","risk_level","risk_factors","success"])
    print("P1-2b smoketest: OK")
except Exception as e:
    print("P1-2b test failed:", e)


P1-2b test result: {'is_impersonation': True, 'confidence': 0.842, 'risk_level': 'high', 'detected_brands': ['paypal'], 'risk_factors': {'typosquatting_matches': [], 'risk_keywords': ['login', 'secure'], 'ml_probability': 0.34, 'tokens': ['paypal', 'secure', 'login', 'info']}, 'reasons': ['brand keyword present', 'no typo candidate', 'ml_probability contribution=0.102'], 'success': True}
P1-2b smoketest: OK


In [12]:
# === Cell NEW.TEST: Unit & handoff smoke test for `_execute_tool_with_trace` (hardened) ===
from typing import Any, Dict
from pathlib import Path
import json, pickle

# --- Fallback stub: define _call_tool if upstream cell wasn't executed ---
if "_call_tool" not in globals():
    def _call_tool(tool_name: str, **kwargs) -> Dict[str, Any]:
        fn = globals().get(tool_name)
        if fn is None:
            return {"error": f"Tool `{tool_name}` not available"}
        try:
            res = fn(**kwargs)
            return res if isinstance(res, dict) else {"value": res}
        except Exception as e:
            return {"error": f"{tool_name} failed: {type(e).__name__}: {e}"}

class _SimpleTrace:
    def __init__(self):
        self.records = []
    def add_tool_trace(self, **kwargs):
        self.records.append(kwargs)

# A tiny demo tool discoverable by `_call_tool` (globals lookup)
def demo_tool(x: int, y: int) -> Dict[str, Any]:
    return {"sum": x + y, "diff": x - y, "inputs": {"x": x, "y": y}}

# ---- run ----
_tr = _SimpleTrace()
_res = _execute_tool_with_trace("demo_tool", {"x": 7, "y": 3}, trace=_tr)

print("[UNIT] result:", _res)
print("[UNIT] trace-count:", len(_tr.records))
print("[UNIT] trace-sample:", _tr.records[0] if _tr.records else {})

# --- Optional handoff smoke (non-invasive, guarded) ---
smoke_dir = globals().get("HANDOFF_DIR", None)
if smoke_dir:
    smoke_path = Path(smoke_dir) / "04-4_smoketest_tool_trace.pkl"
    try:
        with open(smoke_path, "wb") as f:
            pickle.dump({
                "function": "_execute_tool_with_trace",
                "trace_count": len(_tr.records),
                "sample_output": _res
            }, f)
        print(f"[HANDOFF] wrote smoke artifact -> {smoke_path}")
    except Exception as e:
        print(f"[HANDOFF] skip (write error): {e}")
else:
    print("[HANDOFF] skip: HANDOFF_DIR undefined")


[UNIT] result: {'sum': 10, 'diff': 4, 'inputs': {'x': 7, 'y': 3}, 'success': True}
[UNIT] trace-count: 1
[UNIT] trace-sample: {'tool_name': 'demo_tool', 'inputs': {'x': 7, 'y': 3}, 'output': {'sum': 10, 'diff': 4, 'inputs': {'x': 7, 'y': 3}, 'success': True}, 'duration': 6.4373016357421875e-06}
[HANDOFF] wrote smoke artifact -> artifacts/2025-10-22_074213/handoff/04-4_smoketest_tool_trace.pkl


In [13]:
print("== preflight probe ==")
print("has AgentState models?:", 'PhishingDetectionResult' in globals())
print("has PhishingDetectionAgent?:", 'PhishingDetectionAgent' in globals())
print("has evaluate_domain?:", 'evaluate_domain' in globals())
print("has evaluate_domain_fixed?:", 'evaluate_domain_fixed' in globals())

# セル6の終端で出るはずのメッセージが出ているか（実行ログを目視確認）
print("※ セル6実行ログに '✅ LangGraph-based PhishingDetectionAgent is defined' が出ているか確認してください。")


== preflight probe ==
has AgentState models?: True
has PhishingDetectionAgent?: True
has evaluate_domain?: True
has evaluate_domain_fixed?: False
※ セル6実行ログに '✅ LangGraph-based PhishingDetectionAgent is defined' が出ているか確認してください。


In [14]:
print("[probe] has PhishingDetectionAgent:", 'PhishingDetectionAgent' in globals())
print("[probe] has evaluate_domain:", 'evaluate_domain' in globals())
print("[probe] has evaluate_domain_fixed:", 'evaluate_domain_fixed' in globals())

[probe] has PhishingDetectionAgent: True
[probe] has evaluate_domain: True
[probe] has evaluate_domain_fixed: False


In [15]:
# === Cell 06-D-ALIAS: Safe unification (fixed が無ければフォールバック) ===
print("=" * 70)
print("🎯 関数統一設定（安全版）")
print("=" * 70)

# evaluate_domain: *_fixed があれば採用、無ければ既存を使う
if 'evaluate_domain_fixed' in globals():
    evaluate_domain = evaluate_domain_fixed
    print("  ✅ evaluate_domain → evaluate_domain_fixed を使用")
elif 'evaluate_domain' in globals():
    print("  ✅ evaluate_domain → 既存版を使用")
else:
    raise RuntimeError("evaluate_domain / evaluate_domain_fixed が見つかりません。定義セルを先に実行してください。")

# process_domains_batch: *_fixed があれば採用、無ければ既存を使う
if 'process_domains_batch_fixed' in globals():
    process_domains_batch = process_domains_batch_fixed
    print("  ✅ process_domains_batch → process_domains_batch_fixed を使用")
elif 'process_domains_batch' in globals():
    print("  ✅ process_domains_batch → 既存版を使用")
else:
    # どちらも無ければ evaluate_domain にフォールバックで1本ずつ処理する簡易版を定義
    print("  ⚠️ process_domains_batch が未定義のため、簡易フォールバックを定義")
    def process_domains_batch(domains, probs=None, cfg=None, max_concurrent=10):
        probs = probs or [0.0] * len(domains)
        return [evaluate_domain(d, float(p), cfg=cfg or {}) for d, p in zip(domains, probs)]

# AI_AGENT の互換ハンドリング（存在するものを採用）
if 'FIXED_AI_AGENT' in globals():
    AI_AGENT = FIXED_AI_AGENT
    print("  ✅ AI_AGENT → FIXED_AI_AGENT を使用")
elif 'AI_AGENT' in globals():
    print("  ✅ AI_AGENT → 既存版を使用")
else:
    print("  ⚠️ AI_AGENT が未定義（evaluate_domain に依存動作するため続行可能）")

print("📋 定義状況チェック:")
for func in ['evaluate_domain', 'evaluate_domain_fixed',
             'process_domains_batch', 'process_domains_batch_fixed',
             'AI_AGENT', 'FIXED_AI_AGENT']:
    if func in globals():
        print(f"  • {func}: 定義あり")
    else:
        print(f"  • {func}: （未定義）")

print("✅ 安全な関数統一が完了しました。")

🎯 関数統一設定（安全版）
  ✅ evaluate_domain → 既存版を使用
  ⚠️ process_domains_batch が未定義のため、簡易フォールバックを定義
  ⚠️ AI_AGENT が未定義（evaluate_domain に依存動作するため続行可能）
📋 定義状況チェック:
  • evaluate_domain: 定義あり
  • evaluate_domain_fixed: （未定義）
  • process_domains_batch: 定義あり
  • process_domains_batch_fixed: （未定義）
  • AI_AGENT: （未定義）
  • FIXED_AI_AGENT: （未定義）
✅ 安全な関数統一が完了しました。


In [16]:
# === Cell 07: Save output handoff (SAFE, pickle-friendly) ===
import os, pickle, json, warnings, datetime, types, inspect, pickle

# 必要に応じて既存変数（RUN_ID/HANDOFF_DIR/IN_HANDOFF_PKL など）が上セルで定義済みである前提
# OUT_HANDOFF_PKL はこのセルで定義します（上流と同じ命名を踏襲）
HANDOFF_DIR = HANDOFF_DIR if 'HANDOFF_DIR' in globals() else os.path.join("artifacts", RUN_ID, "handoff")
os.makedirs(HANDOFF_DIR, exist_ok=True)
OUT_HANDOFF_PKL = os.path.join(HANDOFF_DIR, "04-4_agent_implementation.pkl")

def _is_pickleable(obj) -> bool:
    try:
        pickle.dumps(obj)
        return True
    except Exception:
        return False

def _ref_info(obj):
    """関数/クラス/グラフなど非ピクル化オブジェクトの軽量参照情報に変換"""
    info = {
        "_nonserializable": True,
        "type": type(obj).__name__,
        "repr": repr(obj)[:240],
    }
    # import可能なcallableなら __module__/__name__ を残す
    if callable(obj):
        info["callable"] = True
        info["module"] = getattr(obj, "__module__", None)
        info["name"] = getattr(obj, "__name__", None)
    # LangGraph の compiled graph っぽいものは型名だけ強調
    if type(obj).__name__.lower().find("compiledstategraph") >= 0:
        info["kind"] = "langgraph_compiled_graph"
    return info

# ---- 1) allowlist: 素のデータだけそのまま保存（ここに基本的に必要なキーを列挙） ----
exports = {}
allowlist_names = [
    "cfg",                 # dict (config)
    "brand_keywords",      # list or dict
    "cert_full_info_map",  # dict
    "RUN_ID",              # str
]
for name in allowlist_names:
    if name in globals():
        val = globals()[name]
        # 念のためpickle可能性チェック
        if _is_pickleable(val):
            exports[name] = val
        else:
            exports[name] = _ref_info(val)

# ---- 2) 参照メタ：関数やグラフオブジェクトの“所在”だけを書く（本体は保存しない） ----
# 04-3 側の成果（tools 等）を参照したい場合はファイル名などのメタを残す
# 例）IN_HANDOFF_PKL_043 等が上流セルにあれば利用。無ければ None。
exports["meta"] = {
    "timestamp": datetime.datetime.utcnow().isoformat() + "Z",
    "module": "04-4_agent_implementation",
    "note": "Functions/graphs are not pickled. Use runtime imports.",
    "with_tools": False,                # ツール関数はpickleしない方針
    "tools_from": globals().get("IN_HANDOFF_PKL_043", None),  # 04-3のpklがあるなら参照メタだけ
    "primary_type": "dict",
}

# ---- 3) 任意：存在すれば評価エイリアス名だけ（中身は保存しない） ----
#   evaluate_domain / evaluate_domain_fixed / AI_AGENT / PHISHING_DETECTION_GRAPH 等はpickleしない
name_refs = ["evaluate_domain_fixed", "evaluate_domain", "AI_AGENT", "PHISHING_DETECTION_GRAPH"]
for name in name_refs:
    if name in globals():
        exports[f"{name}__ref"] = _ref_info(globals()[name])

# ---- 4) 念のため：その他 “保存したい変数” を追加したいときはここに個別追記（必ず _is_pickleable で防御） ----
# 例）統計値や小さめの DataFrame など
# if "stats" in globals() and _is_pickleable(stats): exports["stats"] = stats

# ---- 5) 最後に pickle 保存（ここで失敗しない構成になっている） ----
with open(OUT_HANDOFF_PKL, "wb") as f:
    pickle.dump(exports, f)

print(f"[04-4] saved handoff -> {OUT_HANDOFF_PKL}")
print("[04-4] exported keys:", sorted(exports.keys()))
# === Cell 07: Save output handoff (SAFE, pickle-friendly) ===
import os, pickle, json, warnings, datetime, types, inspect, pickle

# 必要に応じて既存変数（RUN_ID/HANDOFF_DIR/IN_HANDOFF_PKL など）が上セルで定義済みである前提
# OUT_HANDOFF_PKL はこのセルで定義します（上流と同じ命名を踏襲）
HANDOFF_DIR = HANDOFF_DIR if 'HANDOFF_DIR' in globals() else os.path.join("artifacts", RUN_ID, "handoff")
os.makedirs(HANDOFF_DIR, exist_ok=True)
OUT_HANDOFF_PKL = os.path.join(HANDOFF_DIR, "04-4_agent_implementation.pkl")

def _is_pickleable(obj) -> bool:
    try:
        pickle.dumps(obj)
        return True
    except Exception:
        return False

def _ref_info(obj):
    """関数/クラス/グラフなど非ピクル化オブジェクトの軽量参照情報に変換"""
    info = {
        "_nonserializable": True,
        "type": type(obj).__name__,
        "repr": repr(obj)[:240],
    }
    # import可能なcallableなら __module__/__name__ を残す
    if callable(obj):
        info["callable"] = True
        info["module"] = getattr(obj, "__module__", None)
        info["name"] = getattr(obj, "__name__", None)
    # LangGraph の compiled graph っぽいものは型名だけ強調
    if type(obj).__name__.lower().find("compiledstategraph") >= 0:
        info["kind"] = "langgraph_compiled_graph"
    return info

# ---- 1) allowlist: 素のデータだけそのまま保存（ここに基本的に必要なキーを列挙） ----
exports = {}
allowlist_names = [
    "cfg",                 # dict (config)
    "brand_keywords",      # list or dict
    "cert_full_info_map",  # dict
    "RUN_ID",              # str
]
for name in allowlist_names:
    if name in globals():
        val = globals()[name]
        # 念のためpickle可能性チェック
        if _is_pickleable(val):
            exports[name] = val
        else:
            exports[name] = _ref_info(val)

# ---- 2) 参照メタ：関数やグラフオブジェクトの“所在”だけを書く（本体は保存しない） ----
# 04-3 側の成果（tools 等）を参照したい場合はファイル名などのメタを残す
# 例）IN_HANDOFF_PKL_043 等が上流セルにあれば利用。無ければ None。
exports["meta"] = {
    "timestamp": datetime.datetime.utcnow().isoformat() + "Z",
    "module": "04-4_agent_implementation",
    "note": "Functions/graphs are not pickled. Use runtime imports.",
    "with_tools": False,                # ツール関数はpickleしない方針
    "tools_from": globals().get("IN_HANDOFF_PKL_043", None),  # 04-3のpklがあるなら参照メタだけ
    "primary_type": "dict",
}

# ---- 3) 任意：存在すれば評価エイリアス名だけ（中身は保存しない） ----
#   evaluate_domain / evaluate_domain_fixed / AI_AGENT / PHISHING_DETECTION_GRAPH 等はpickleしない
name_refs = ["evaluate_domain_fixed", "evaluate_domain", "AI_AGENT", "PHISHING_DETECTION_GRAPH"]
for name in name_refs:
    if name in globals():
        exports[f"{name}__ref"] = _ref_info(globals()[name])

# ---- 4) 念のため：その他 “保存したい変数” を追加したいときはここに個別追記（必ず _is_pickleable で防御） ----
# 例）統計値や小さめの DataFrame など
# if "stats" in globals() and _is_pickleable(stats): exports["stats"] = stats

# ---- 5) 最後に pickle 保存（ここで失敗しない構成になっている） ----
with open(OUT_HANDOFF_PKL, "wb") as f:
    pickle.dump(exports, f)

print(f"[04-4] saved handoff -> {OUT_HANDOFF_PKL}")
print("[04-4] exported keys:", sorted(exports.keys()))
# === Cell 07: Save output handoff (SAFE, pickle-friendly) ===
import os, pickle, json, warnings, datetime, types, inspect, pickle

# 必要に応じて既存変数（RUN_ID/HANDOFF_DIR/IN_HANDOFF_PKL など）が上セルで定義済みである前提
# OUT_HANDOFF_PKL はこのセルで定義します（上流と同じ命名を踏襲）
HANDOFF_DIR = HANDOFF_DIR if 'HANDOFF_DIR' in globals() else os.path.join("artifacts", RUN_ID, "handoff")
os.makedirs(HANDOFF_DIR, exist_ok=True)
OUT_HANDOFF_PKL = os.path.join(HANDOFF_DIR, "04-4_agent_implementation.pkl")

def _is_pickleable(obj) -> bool:
    try:
        pickle.dumps(obj)
        return True
    except Exception:
        return False

def _ref_info(obj):
    """関数/クラス/グラフなど非ピクル化オブジェクトの軽量参照情報に変換"""
    info = {
        "_nonserializable": True,
        "type": type(obj).__name__,
        "repr": repr(obj)[:240],
    }
    # import可能なcallableなら __module__/__name__ を残す
    if callable(obj):
        info["callable"] = True
        info["module"] = getattr(obj, "__module__", None)
        info["name"] = getattr(obj, "__name__", None)
    # LangGraph の compiled graph っぽいものは型名だけ強調
    if type(obj).__name__.lower().find("compiledstategraph") >= 0:
        info["kind"] = "langgraph_compiled_graph"
    return info

# ---- 1) allowlist: 素のデータだけそのまま保存（ここに基本的に必要なキーを列挙） ----
exports = {}
allowlist_names = [
    "cfg",                 # dict (config)
    "brand_keywords",      # list or dict
    "cert_full_info_map",  # dict
    "RUN_ID",              # str
]
for name in allowlist_names:
    if name in globals():
        val = globals()[name]
        # 念のためpickle可能性チェック
        if _is_pickleable(val):
            exports[name] = val
        else:
            exports[name] = _ref_info(val)

# ---- 2) 参照メタ：関数やグラフオブジェクトの“所在”だけを書く（本体は保存しない） ----
# 04-3 側の成果（tools 等）を参照したい場合はファイル名などのメタを残す
# 例）IN_HANDOFF_PKL_043 等が上流セルにあれば利用。無ければ None。
exports["meta"] = {
    "timestamp": datetime.datetime.utcnow().isoformat() + "Z",
    "module": "04-4_agent_implementation",
    "note": "Functions/graphs are not pickled. Use runtime imports.",
    "with_tools": False,                # ツール関数はpickleしない方針
    "tools_from": globals().get("IN_HANDOFF_PKL_043", None),  # 04-3のpklがあるなら参照メタだけ
    "primary_type": "dict",
}

# ---- 3) 任意：存在すれば評価エイリアス名だけ（中身は保存しない） ----
#   evaluate_domain / evaluate_domain_fixed / AI_AGENT / PHISHING_DETECTION_GRAPH 等はpickleしない
name_refs = ["evaluate_domain_fixed", "evaluate_domain", "AI_AGENT", "PHISHING_DETECTION_GRAPH"]
for name in name_refs:
    if name in globals():
        exports[f"{name}__ref"] = _ref_info(globals()[name])

# ---- 4) 念のため：その他 “保存したい変数” を追加したいときはここに個別追記（必ず _is_pickleable で防御） ----
# 例）統計値や小さめの DataFrame など
# if "stats" in globals() and _is_pickleable(stats): exports["stats"] = stats

# ---- 5) 最後に pickle 保存（ここで失敗しない構成になっている） ----
with open(OUT_HANDOFF_PKL, "wb") as f:
    pickle.dump(exports, f)

print(f"[04-4] saved handoff -> {OUT_HANDOFF_PKL}")
print("[04-4] exported keys:", sorted(exports.keys()))
import os, pickle, json, warnings, datetime, types, inspect, pickle

# 必要に応じて既存変数（RUN_ID/HANDOFF_DIR/IN_HANDOFF_PKL など）が上セルで定義済みである前提
# OUT_HANDOFF_PKL はこのセルで定義します（上流と同じ命名を踏襲）
HANDOFF_DIR = HANDOFF_DIR if 'HANDOFF_DIR' in globals() else os.path.join("artifacts", RUN_ID, "handoff")
os.makedirs(HANDOFF_DIR, exist_ok=True)
OUT_HANDOFF_PKL = os.path.join(HANDOFF_DIR, "04-4_agent_implementation.pkl")

def _is_pickleable(obj) -> bool:
    try:
        pickle.dumps(obj)
        return True
    except Exception:
        return False

def _ref_info(obj):
    """関数/クラス/グラフなど非ピクル化オブジェクトの軽量参照情報に変換"""
    info = {
        "_nonserializable": True,
        "type": type(obj).__name__,
        "repr": repr(obj)[:240],
    }
    # import可能なcallableなら __module__/__name__ を残す
    if callable(obj):
        info["callable"] = True
        info["module"] = getattr(obj, "__module__", None)
        info["name"] = getattr(obj, "__name__", None)
    # LangGraph の compiled graph っぽいものは型名だけ強調
    if type(obj).__name__.lower().find("compiledstategraph") >= 0:
        info["kind"] = "langgraph_compiled_graph"
    return info

# ---- 1) allowlist: 素のデータだけそのまま保存（ここに基本的に必要なキーを列挙） ----
exports = {}
allowlist_names = [
    "cfg",                 # dict (config)
    "brand_keywords",      # list or dict
    "cert_full_info_map",  # dict
    "RUN_ID",              # str
]
for name in allowlist_names:
    if name in globals():
        val = globals()[name]
        # 念のためpickle可能性チェック
        if _is_pickleable(val):
            exports[name] = val
        else:
            exports[name] = _ref_info(val)

# ---- 2) 参照メタ：関数やグラフオブジェクトの“所在”だけを書く（本体は保存しない） ----
# 04-3 側の成果（tools 等）を参照したい場合はファイル名などのメタを残す
# 例）IN_HANDOFF_PKL_043 等が上流セルにあれば利用。無ければ None。
exports["meta"] = {
    "timestamp": datetime.datetime.utcnow().isoformat() + "Z",
    "module": "04-4_agent_implementation",
    "note": "Functions/graphs are not pickled. Use runtime imports.",
    "with_tools": False,                # ツール関数はpickleしない方針
    "tools_from": globals().get("IN_HANDOFF_PKL_043", None),  # 04-3のpklがあるなら参照メタだけ
    "primary_type": "dict",
}

# ---- 3) 任意：存在すれば評価エイリアス名だけ（中身は保存しない） ----
#   evaluate_domain / evaluate_domain_fixed / AI_AGENT / PHISHING_DETECTION_GRAPH 等はpickleしない
name_refs = ["evaluate_domain_fixed", "evaluate_domain", "AI_AGENT", "PHISHING_DETECTION_GRAPH"]
for name in name_refs:
    if name in globals():
        exports[f"{name}__ref"] = _ref_info(globals()[name])

# ---- 4) 念のため：その他 “保存したい変数” を追加したいときはここに個別追記（必ず _is_pickleable で防御） ----
# 例）統計値や小さめの DataFrame など
# if "stats" in globals() and _is_pickleable(stats): exports["stats"] = stats

# ---- 5) 最後に pickle 保存（ここで失敗しない構成になっている） ----
with open(OUT_HANDOFF_PKL, "wb") as f:
    pickle.dump(exports, f)

print(f"[04-4] saved handoff -> {OUT_HANDOFF_PKL}")
print("[04-4] exported keys:", sorted(exports.keys()))


[04-4] saved handoff -> artifacts/2025-10-22_074213/handoff/04-4_agent_implementation.pkl
[04-4] exported keys: ['RUN_ID', 'cfg', 'evaluate_domain__ref', 'meta']
[04-4] saved handoff -> artifacts/2025-10-22_074213/handoff/04-4_agent_implementation.pkl
[04-4] exported keys: ['RUN_ID', 'cfg', 'evaluate_domain__ref', 'meta']
[04-4] saved handoff -> artifacts/2025-10-22_074213/handoff/04-4_agent_implementation.pkl
[04-4] exported keys: ['RUN_ID', 'cfg', 'evaluate_domain__ref', 'meta']
[04-4] saved handoff -> artifacts/2025-10-22_074213/handoff/04-4_agent_implementation.pkl
[04-4] exported keys: ['RUN_ID', 'cfg', 'evaluate_domain__ref', 'meta']


In [17]:
# === Cell 08: Quick self-check (no logic change) ===
print("evaluate_domain:", "OK" if "evaluate_domain" in globals() else "MISSING")
print("process_domains_batch:", "OK" if "process_domains_batch" in globals() else "MISSING")
print("PHISHING_DETECTION_GRAPH:", "OK" if "PHISHING_DETECTION_GRAPH" in globals() else "MISSING")
print("PhishingDetectionAgent:", "OK" if "PhishingDetectionAgent" in globals() else "MISSING")


evaluate_domain: OK
process_domains_batch: OK
PHISHING_DETECTION_GRAPH: MISSING
PhishingDetectionAgent: OK


In [18]:
# === P1-1: process_domains_batch_fixed (restored minimal, fault-tolerant) ===
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Dict, Any, Optional
import time, traceback, os, pickle

SUSPICIOUS_KEYWORDS = {'secure','login','verify','update','account','wallet','support','confirm','bank','invoice','payment'}

def _risk_level_from_confidence(conf: float) -> str:
    if conf >= 0.85: return "critical"
    if conf >= 0.70: return "high"
    if conf >= 0.50: return "medium-high"
    if conf >= 0.30: return "medium"
    return "low"

def _normalize_result(domain, prob, res):
    # Normalize various possible return types into unified dict
    if hasattr(res, "to_dict"):
        try:
            res = res.to_dict()
        except Exception:
            pass
    elif hasattr(res, "model_dump"):
        try:
            res = res.model_dump()
        except Exception:
            pass
    elif hasattr(res, "dict"):
        try:
            res = res.dict()
        except Exception:
            pass
    if not isinstance(res, dict):
        res = {}
    out = {
        "domain": domain,
        "ml_probability": float(prob) if prob is not None else None,
        "is_phishing": bool(res.get("is_phishing", False)),
        "confidence": float(res.get("confidence", res.get("ai_confidence", 0.0) or 0.0)),
        "risk_level": res.get("risk_level", None),
        "risk_factors": res.get("risk_factors", {}),
        "reasoning": res.get("reasoning", ""),
        "success": bool(res.get("success", True)),
    }
    if not out["risk_level"]:
        out["risk_level"] = _risk_level_from_confidence(out["confidence"])
    if isinstance(res, dict) and res.get("mock"):
        out.setdefault("risk_factors", {})["mock"] = True
    return out

def process_domains_batch_fixed(
    domains: List[str],
    probs: Optional[List[float]] = None,
    cfg: Optional[Dict[str, Any]] = None,
    max_concurrent: int = 8,
    agent: Any = None,
    mock: bool = False,
) -> List[Dict[str, Any]]:
    """
    Fault-tolerant batch processor for domain analysis.

    Args:
        domains: list of domain strings
        probs: list of ML probabilities (0-1). If None, zeros are used.
        cfg: optional config dict (kept for compatibility)
        max_concurrent: degree of concurrency
        agent: optional object with .evaluate(domain, prob)
        mock: True -> deterministic, I/O-free evaluation (for tests)
    Returns:
        List[dict]: unified results per domain
    """
    if probs is None:
        probs = [0.0] * len(domains)

    n = min(len(domains), len(probs))
    domains = list(domains)[:n]
    probs = list(probs)[:n]

    g = globals()
    evaluator = None
    if "evaluate_domain_fixed" in g and callable(g["evaluate_domain_fixed"]):
        evaluator = g["evaluate_domain_fixed"]
    elif "evaluate_domain" in g and callable(g["evaluate_domain"]):
        evaluator = g["evaluate_domain"]
    elif agent is not None and hasattr(agent, "evaluate") and callable(agent.evaluate):
        evaluator = lambda d, p: agent.evaluate(d, p)  # noqa
    else:
        evaluator = None

    def _mock_eval(domain, prob):
        lower = (domain or "").lower()
        score = 0.0
        kw = [k for k in SUSPICIOUS_KEYWORDS if k in lower]
        if kw:
            score += 0.4 + 0.05 * min(len(kw), 3)
        if "-" in lower:
            score += 0.1
        if any(ch.isdigit() for ch in lower):
            score += 0.05
        score += float(prob or 0.0) * 0.3
        score = max(0.0, min(score, 0.99))
        return {
            "is_phishing": score >= 0.5,
            "confidence": score,
            "risk_factors": {"keywords": kw, "ml_probability": float(prob or 0.0)},
            "reasoning": "Mock evaluation (no external I/O).",
            "success": True,
            "mock": True,
        }

    def _safe_eval(domain, prob):
        try:
            if mock or evaluator is None:
                res = _mock_eval(domain, prob)
            else:
                # Support both evaluator(domain, prob) and evaluator(domain=..., ml_probability=...)
                try:
                    res = evaluator(domain=domain, ml_probability=prob)
                except TypeError:
                    res = evaluator(domain, prob)
            out = _normalize_result(domain, prob, res)
            if out.get("success") is None:
                out["success"] = True
            return out
        except Exception as e:  # pragma: no cover
            tb = traceback.format_exc(limit=1)
            return {
                "domain": domain,
                "ml_probability": float(prob or 0.0),
                "is_phishing": False,
                "confidence": 0.0,
                "risk_level": "low",
                "risk_factors": {"error": str(e), "where": "process_domains_batch_fixed", "trace": tb},
                "reasoning": "Exception during evaluation; returning safe default.",
                "success": False,
            }

    results = [None] * n
    with ThreadPoolExecutor(max_workers=max(1, int(max_concurrent or 1))) as ex:
        futs = {ex.submit(_safe_eval, d, p): i for i, (d, p) in enumerate(zip(domains, probs))}
        for fut in as_completed(futs):
            i = futs[fut]
            results[i] = fut.result()

    # Optional handoff artifacts (if globals exist)
    try:
        run_id = g.get("RUN_ID", None)
        handoff_dir = g.get("HANDOFF_DIR", None)
        if run_id and handoff_dir:
            os.makedirs(handoff_dir, exist_ok=True)
            with open(os.path.join(handoff_dir, "04-4_smoketest_process_domains_batch_fixed.pkl"), "wb") as f:
                pickle.dump({"n": n, "ts": time.time()}, f)
            with open(os.path.join(handoff_dir, "04-4_smoketest_process_domains_batch_fixed_results.pkl"), "wb") as f:
                pickle.dump(results, f)
    except Exception:
        pass

    return results

In [19]:
# === P1-1 TEST (mock, no external I/O) ===
_test_domains = ["paypal-secure-login.info", "example.com"]
_test_probs   = [0.34, 0.05]
_test_res = process_domains_batch_fixed(_test_domains, _test_probs, mock=True, max_concurrent=2)
print("P1-1 test results:", _test_res)
assert len(_test_res) == 2 and all("domain" in r for r in _test_res), "Unexpected test output shape"
print("P1-1 smoketest: OK")

P1-1 test results: [{'domain': 'paypal-secure-login.info', 'ml_probability': 0.34, 'is_phishing': True, 'confidence': 0.702, 'risk_level': 'high', 'risk_factors': {'keywords': ['login', 'secure'], 'ml_probability': 0.34, 'mock': True}, 'reasoning': 'Mock evaluation (no external I/O).', 'success': True}, {'domain': 'example.com', 'ml_probability': 0.05, 'is_phishing': False, 'confidence': 0.015, 'risk_level': 'low', 'risk_factors': {'keywords': [], 'ml_probability': 0.05, 'mock': True}, 'reasoning': 'Mock evaluation (no external I/O).', 'success': True}]
P1-1 smoketest: OK


In [20]:
# === P1-1 ALIAS (keep public name stable) ===
try:
    process_domains_batch = process_domains_batch_fixed
except Exception as _e:
    print("Alias setup failed:", _e)

In [21]:
# === C-02 FixedPhishingDetectionAgent (04-4) ===
# NOTE: Appended by automation. Minimal-diff restoration from original 03 notebook design.
# - Reads artifacts/{RUN_ID}/handoff/04-3_llm_tools_setup_with_tools.pkl
# - Provides FixedPhishingDetectionAgent with analyze(), _execute_tool(), and helpers
# - Guarantees Structured Output keys: is_phishing, confidence, risk_factors, reasoning
# - Safe fallbacks when tools aren't present yet (explicitly mocked)
from dataclasses import dataclass, field
from typing import Dict, Any, List, Tuple, Optional
import os, json, pickle, time, math, re, datetime as _dt

# ---------- handoff loader (from 04-3) ----------
def _load_llm_setup(run_id: Optional[str]=None) -> Dict[str, Any]:
    run_id = run_id or os.getenv("RUN_ID") or _dt.datetime.now().strftime("%Y%m%d_%H%M%S")
    pkl_path = os.path.join("/mnt/data", "artifacts", run_id, "handoff", "04-3_llm_tools_setup_with_tools.pkl")
    if os.path.exists(pkl_path):
        with open(pkl_path, "rb") as f:
            state = pickle.load(f) or {}
        return state
    # fallback: empty skeleton
    return {
        "provider": None, "base_url": None, "model": None, "llm": {},
        "brand_keywords": [], "brand_keywords_cfg": {},
        "paths": {}, "run_id": run_id, "timestamp": _dt.datetime.now().isoformat(),
        "required_stats": [], "required_top": [], "present_stats": [], "present_top": [],
        "missing_stats": [], "missing_top": [], "stats_keys": [], "top_level_keys": []
    }

# ---------- tiny utility: entropy ----------
def _entropy(text: str) -> float:
    if not text:
        return 0.0
    probs = [text.count(c)/len(text) for c in set(text)]
    return -sum(p*math.log(p, 2) for p in probs if p > 0)

# ---------- Agent Implementation ----------
@dataclass
class FixedPhishingDetectionAgent:
    provider: Optional[str] = None
    base_url: Optional[str] = None
    model: Optional[str] = None
    brand_keywords: List[str] = field(default_factory=list)
    high_risk_words: List[str] = field(default_factory=lambda: [
        "secure","verify","update","confirm","account","login","signin","validation","verification","official"
    ])
    trace_enabled: bool = False

    @classmethod
    def from_handoff(cls, run_id: Optional[str]=None) -> "FixedPhishingDetectionAgent":
        s = _load_llm_setup(run_id)
        # if brand list empty, seed with paper examples (paypal, mercari, ledger) as a conservative default
        seed_brands = ["paypal","mercari","ledger"]
        bk = s.get("brand_keywords") or seed_brands
        return cls(
            provider=s.get("provider"), base_url=s.get("base_url"), model=s.get("model"),
            brand_keywords=bk
        )

    # ---- internal helpers ----
    def _check_obvious_phishing(self, domain: str) -> Tuple[bool, List[str], float]:
        domain_lower = domain.lower()
        detected = []
        boost = 0.0
        # brand hit
        for b in self.brand_keywords:
            try:
                bl = b.lower()
            except Exception:
                continue
            if bl and bl in domain_lower:
                detected.append(b)
                boost = max(boost, 0.15)
        # high-risk words
        for w in self.high_risk_words:
            if w in domain_lower:
                boost = max(boost, 0.10)
        # obvious heuristic
        is_obvious = (len(detected) > 0) and any(w in domain_lower for w in ["secure","login","update","verify"])
        if is_obvious:
            boost = max(boost, 0.35)
        return is_obvious, detected, boost

    def _create_aggressive_prompt(self, domain: str, ml_probability: float, is_obvious: bool) -> str:
        return (
            "You are a security analyst. Determine if the domain is a phishing site. "
            f"Domain: {domain}\n"
            f"ML Probability: {ml_probability:.3f}\n"
            f"Obvious-Flag: {is_obvious}\n"
            "Return structured JSON with fields: is_phishing (bool), confidence (0..1), "
            "risk_factors (object), reasoning (short)."
        )

    def _parse_response_aggressive(self, response: str) -> Dict[str, Any]:
        # Placeholder parser: expect JSON-like substrings; fallback to conservative result
        try:
            import json
            data = json.loads(response)
            # ensure keys
            return {
                "is_phishing": bool(data.get("is_phishing", False)),
                "confidence": max(0.0, min(1.0, float(data.get("confidence", 0.5)))),
                "risk_factors": dict(data.get("risk_factors", {})),
                "reasoning": str(data.get("reasoning", ""))[:2000],
            }
        except Exception:
            return {"is_phishing": False, "confidence": 0.5, "risk_factors": {}, "reasoning": "parse-fallback"}

    # tool dispatcher with graceful fallback
    def _execute_tool(self, tool_name: str, inputs: Dict[str, Any]) -> Dict[str, Any]:
        t0 = time.time()
        out = {"tool": tool_name, "ok": True, "inputs": inputs, "output": None, "latency_ms": None}
        try:
            if tool_name == "short_domain_analysis":
                # try global implementation
                if "short_domain_analysis" in globals():
                    out["output"] = globals()["short_domain_analysis"](inputs["domain"], inputs.get("ml_probability", 0.0))
                else:
                    # fallback quick heuristic
                    domain = inputs["domain"]
                    parts = domain.split(".")
                    without_tld = ".".join(parts[:-1]) if len(parts) > 1 else domain
                    ent = _entropy(without_tld)
                    hyphens = domain.count("-")
                    digit_ratio = sum(c.isdigit() for c in without_tld) / max(1, len(without_tld))
                    factors = {}
                    reasons = []
                    conf = 0.0
                    if len(without_tld) <= 10: 
                        factors["short_name"] = True; conf = max(conf, 0.30); reasons.append("短いドメイン名")
                    if ent > 4.0: 
                        factors["high_entropy"] = True; conf = max(conf, 0.55); reasons.append(f"高エントロピー {ent:.2f}")
                    if hyphens > 2: 
                        factors["many_hyphens"] = True; conf = max(conf, 0.45); reasons.append(f"ハイフン多 {hyphens}")
                    if digit_ratio > 0.3:
                        factors["digit_ratio_high"] = True; conf = max(conf, 0.40); reasons.append(f"数字割合 {digit_ratio:.2f}")
                    out["output"] = {
                        "is_phishing": conf >= 0.5,
                        "confidence": conf,
                        "risk_factors": factors,
                        "reasoning": "; ".join(reasons) or "no-issues"
                    }
            elif tool_name == "brand_check":
                domain = inputs["domain"].lower()
                hits = [b for b in self.brand_keywords if isinstance(b, str) and b.lower() in domain]
                out["output"] = {
                    "is_phishing": len(hits) > 0,
                    "confidence": 0.65 if len(hits) > 0 else 0.3,
                    "risk_factors": {"brand_hits": hits},
                    "reasoning": "brand keywords matched" if hits else "no brand match"
                }
            elif tool_name == "cert_analysis":
                # placeholder; real impl will be connected later
                out["output"] = {
                    "is_phishing": False,
                    "confidence": 0.35,
                    "risk_factors": {"cert_org_missing": True},
                    "reasoning": "mock: org field missing (placeholder)"
                }
            elif tool_name == "context_risk_assessment":
                # aggregate signals from previous results (pass via inputs)
                signals = inputs.get("signals", {})
                conf = max([v.get("confidence", 0.0) for v in signals.values()] + [0.0])
                is_ph = any(v.get("is_phishing", False) for v in signals.values())
                out["output"] = {
                    "is_phishing": is_ph,
                    "confidence": conf,
                    "risk_factors": {"combined": {k: v.get("risk_factors", {}) for k,v in signals.items()}},
                    "reasoning": "combined context risk (placeholder)"
                }
            else:
                out["ok"] = False
                out["output"] = {"error": f"unknown tool {tool_name}"}
        except Exception as e:
            out["ok"] = False
            out["output"] = {"error": str(e)}
        finally:
            out["latency_ms"] = int((time.time() - t0)*1000)
        return out

    # ---- main entry ----
    def analyze(self, domain: str, ml_probability: float, trace: Optional[Dict[str, Any]]=None) -> Dict[str, Any]:
        trace_enabled = bool(trace) or self.trace_enabled
        result = {
            "domain": domain,
            "ml_probability": float(ml_probability),
            "is_phishing": False,
            "confidence": float(ml_probability),
            "risk_factors": {},
            "reasoning": "",
            "detected_brands": [],
            "tools_used": [],
        }
        t_all = time.time()

        is_obvious, brands, boost = self._check_obvious_phishing(domain)
        result["detected_brands"] = brands
        result["confidence"] = max(result["confidence"], min(1.0, result["confidence"] + boost))

        # tool selection policy following paper: <0.2 => force all tools; >=0.2 => selective
        tools = ["brand_check", "short_domain_analysis", "cert_analysis", "context_risk_assessment"]
        if ml_probability >= 0.2:
            tools = ["brand_check", "cert_analysis", "context_risk_assessment"]

        # run tools (brand/short/cert first; context after)
        signals: Dict[str, Dict[str, Any]] = {}
        for t in tools:
            if t == "context_risk_assessment":
                # pass previous signals
                out = self._execute_tool(t, {"domain": domain, "signals": signals})
            else:
                out = self._execute_tool(t, {"domain": domain, "ml_probability": ml_probability})
                # collect
                sig = out.get("output", {}) if out.get("ok") else {}
                signals[t] = sig
            result["tools_used"].append({k: out.get(k) for k in ("tool","ok","latency_ms")})

        # combine signals
        confs = [signals[k].get("confidence", 0.0) for k in signals if isinstance(signals[k], dict)]
        max_conf = max(confs + [result["confidence"]])
        is_ph = any(signals[k].get("is_phishing", False) for k in signals if isinstance(signals[k], dict))
        # heuristics
        if is_obvious or is_ph:
            result["is_phishing"] = True
            result["confidence"] = max(0.7, max_conf)
        else:
            result["is_phishing"] = (max_conf >= 0.8)  # conservative

        # reasoning
        reasons = []
        if brands: reasons.append(f"ブランド一致: {', '.join(brands)}")
        if is_obvious: reasons.append("高リスク語とブランド一致")
        # add top 2 signal reasons
        for name in ["brand_check","short_domain_analysis","cert_analysis"]:
            rs = signals.get(name, {})
            if rs.get("reasoning"): reasons.append(f"{name}: {rs['reasoning']}")
        result["reasoning"] = " / ".join(reasons)[:1500]

        if trace_enabled:
            result["trace_enabled"] = True
            result["tool_signals"] = signals
            result["total_latency_ms"] = int((time.time() - t_all)*1000)

        # ensure Structured Output
        for k in ("is_phishing","confidence","risk_factors","reasoning"):
            if k not in result:
                if k == "is_phishing": result[k] = False
                elif k == "confidence": result[k] = 0.5
                elif k == "risk_factors": result[k] = {}
                elif k == "reasoning": result[k] = ""
        return result

# convenience factory for notebooks
def make_fixed_agent(run_id: Optional[str]=None, trace_enabled: bool=False) -> FixedPhishingDetectionAgent:
    s = _load_llm_setup(run_id)
    agent = FixedPhishingDetectionAgent.from_handoff(run_id)
    agent.trace_enabled = trace_enabled
    return agent

print("[C-02] FixedPhishingDetectionAgent is ready.")

[C-02] FixedPhishingDetectionAgent is ready.


In [22]:
# === C-03b: Attach evaluation candidates into 04-4 handoff (minimal) ===
import warnings, pickle, os
from pathlib import Path
import pandas as pd

# 標準パス解決
try:
    import importlib, run_id_registry as runreg, _compat.paths as paths
    rid = runreg.bootstrap()
    importlib.reload(paths); 
    paths.ensure_roots()
    RUN_ID = paths.RUN_ID or os.environ.get("RUN_ID")
    BASE = paths.compat_base_dirs
    HANDOFF_DIR = Path(BASE["handoff"]); RESULTS_DIR = Path(BASE["results"])
except Exception as e:
    warnings.warn(f"[04-4] compat paths fallback: {e}")
    RUN_ID = os.environ.get("RUN_ID")
    HANDOFF_DIR = Path(f"artifacts/{RUN_ID}/handoff"); RESULTS_DIR = Path(f"artifacts/{RUN_ID}/results")

out_pkl = HANDOFF_DIR / "04-4_agent_implementation.pkl"

# 既存exportsを読み込み（無ければ空）
exports = {}
try:
    if out_pkl.exists() and out_pkl.stat().st_size > 0:
        with open(out_pkl, "rb") as f:
            exports = pickle.load(f) or {}
except Exception as e:
    warnings.warn(f"[04-4] load existing exports failed: {e}; recreate empty")
    exports = {}

def _pick_prob_col(df):
    for c in ["ml_probability","ml_prob","prob","proba","pred_prob","pred_proba","probability","phish_prob","p_phish","score"]:
        if c in getattr(df, "columns", []): 
            return c
    return None

def _try_build_eval_samples():
    for name in ["candidates_df", "eval_df", "evaluation_results", "res_df"]:
        if name in globals():
            df = globals()[name]
            if isinstance(df, pd.DataFrame) and "domain" in df.columns:
                col = _pick_prob_col(df)
                if col:
                    return df[["domain", col]].rename(columns={col: "ml_probability"}).copy()
    # 04-3 handoffのfn_features_dfがあれば流用
    try:
        src43 = HANDOFF_DIR / "04-3_llm_tools_setup_with_tools.pkl"
        if src43.exists() and src43.stat().st_size > 0:
            with open(src43, "rb") as f:
                d43 = pickle.load(f) or {}
            if "fn_features_df" in d43:
                return d43["fn_features_df"]
    except Exception as e:
        warnings.warn(f"[04-4] fn_features_df import failed: {e}")
    return None

eval_df = _try_build_eval_samples()

if eval_df is not None and len(eval_df) > 0:
    if len(eval_df) > 5000:
        eval_df = eval_df.sample(5000, random_state=42).reset_index(drop=True)
    exports["eval_samples"] = eval_df.reset_index(drop=True)
    print(f"[04-4] exports['eval_samples'] set (rows={len(eval_df)})")
else:
    # CSVで場所を教える方式（手掛かりが無い場合はスキップ可）
    try:
        RESULTS_DIR.mkdir(parents=True, exist_ok=True)
        candidates_csv = RESULTS_DIR / "evaluation_candidates.csv"
        if eval_df is not None:
            eval_df.to_csv(candidates_csv, index=False)
            exports.setdefault("meta", {}); exports["meta"].setdefault("paths", {})
            exports["meta"]["paths"]["evaluation_candidates_csv"] = str(candidates_csv)
            print(f"[04-4] meta.paths.evaluation_candidates_csv -> {candidates_csv}")
    except Exception as e:
        warnings.warn(f"[04-4] cannot save candidates_csv: {e}")

# 参照もあれば格納（Mock回避に寄与）
def _ref_info(obj):
    return {"module": getattr(obj, "__module__", None),
            "qualname": getattr(obj, "__qualname__", getattr(obj, "__name__", None))}

for ref_name in ["evaluate_domain", "evaluate_domain_fixed"]:
    if ref_name in globals():
        try:
            exports[f"{ref_name}__ref"] = _ref_info(globals()[ref_name])
        except Exception as e:
            warnings.warn(f"[04-4] cannot set {ref_name}__ref: {e}")

# 上書き保存
with open(out_pkl, "wb") as f:
    pickle.dump(exports, f)
print("[04-4] handoff updated:", out_pkl, "| keys:", sorted(exports.keys()))

[04-4] handoff updated: artifacts/2025-10-22_074213/handoff/04-4_agent_implementation.pkl | keys: ['RUN_ID', 'cfg', 'evaluate_domain__ref', 'meta']


In [ ]:
# === Cell 06-A-ASYNC: analyze_domain_async (minimal restore) ===
import asyncio
from typing import Dict, Any

# 前提: PhishingDetectionAgent / FixedPhishingDetectionAgent / PhishingDetectionResult が既に定義済み

async def _analyze_domain_async_core(agent, domain: str, ml_prob: float, index: int):
    """
    Core logic placeholder:
    - Delegates to agent's internal evaluate_domain/equivalent if available
    - Ensures structured output (dict or PhishingDetectionResult)
    - This is a minimal, non-breaking shim for staged restoration.
    """
    if hasattr(agent, "evaluate_domain_fixed"):
        return await agent.evaluate_domain_fixed(domain=domain, ml_probability=ml_prob, index=index)
    if hasattr(agent, "evaluate_domain"):
        return await agent.evaluate_domain(domain=domain, ml_probability=ml_prob, index=index)
    # フォールバック（構造化）
    return {
        "domain": domain,
        "ml_probability": ml_prob,
        "index": index,
        "is_phishing": None,
        "confidence": None,
        "risk_factors": {},
        "reasoning": "[04-4] No evaluate_domain* found; returned fallback."
    }

def _patch_agent_with_async_method():
    async def analyze_domain_async(self, domain: str, ml_prob: float, index: int):
        try:
            return await _analyze_domain_async_core(self, domain, ml_prob, index)
        except Exception as e:
            return {
                "domain": domain,
                "ml_probability": ml_prob,
                "index": index,
                "error": f"[04-4] analyze_domain_async failed: {e}"
            }

    # 既存クラス双方にアタッチ（Fixed系にも）
    for _cls_name in ("PhishingDetectionAgent", "FixedPhishingDetectionAgent"):
        _cls = globals().get(_cls_name)
        if _cls and not hasattr(_cls, "analyze_domain_async"):
            setattr(_cls, "analyze_domain_async", analyze_domain_async)

_patch_agent_with_async_method()
print("✅ [04-4] analyze_domain_async restored (async method attached)")


In [ ]:
# === Cell 06-A-ASYNC-TEST (Jupyter-safe) ===
import asyncio
try:
    # Running loop exists (Jupyter etc.) -> use top-level await
    _ = asyncio.get_running_loop()
    test_agent = FixedPhishingDetectionAgent() if 'FixedPhishingDetectionAgent' in globals() else PhishingDetectionAgent()
    res = await test_agent.analyze_domain_async("paypal-secure-login.info", 0.34, 0)
    print("[TEST] analyze_domain_async -> OK")
    if isinstance(res, dict):
        print(res)
    else:
        print(getattr(res, "__dict__", res))
except RuntimeError:
    # No running loop -> asyncio.run
    test_agent = FixedPhishingDetectionAgent() if 'FixedPhishingDetectionAgent' in globals() else PhishingDetectionAgent()
    res = asyncio.run(test_agent.analyze_domain_async("paypal-secure-login.info", 0.34, 0))
    print("[TEST] analyze_domain_async -> OK")
    if isinstance(res, dict):
        print(res)
    else:
        print(getattr(res, "__dict__", res))

In [ ]:
# === Cell 06-B-RISK: contextual_risk_assessment (restored/fixed + compat aliases) ===
from typing import Dict, Any

def _safe_get(d: Dict[str, Any], keys, default=None):
    for k in keys:
        if isinstance(k, (list, tuple)):
            cur = d
            ok = True
            for kk in k:
                if isinstance(cur, dict) and kk in cur:
                    cur = cur[kk]
                else:
                    ok = False
                    break
            if ok:
                return cur
        else:
            if isinstance(d, dict) and k in d:
                return d[k]
    return default

def contextual_risk_assessment(domain: str,
                               ml_probability: float,
                               brand_result: Dict[str, Any],
                               cert_result: Dict[str, Any],
                               domain_result: Dict[str, Any]) -> Dict[str, Any]:
    """Return dict: {is_phishing, confidence, risk_factors, reasoning} + compat aliases"""
    ml_prob = float(max(0.0, min(1.0, ml_probability if ml_probability is not None else 0.0)))
    risk_from_ml = 1.0 - ml_prob

    brand_match = bool(_safe_get(brand_result, ["brand_match", "is_brand_imitated", "match"], False))
    brand_name = _safe_get(brand_result, ["brand", "target_brand", "matched_brand"], None)
    typo_score = float(_safe_get(brand_result, ["typosquatting_score", "typo_score"], 0.0) or 0.0)
    risk_brand = 0.0
    if brand_match:
        risk_brand += 0.35
    risk_brand += min(0.25, max(0.0, typo_score) * 0.25)

    org_present = _safe_get(cert_result, ["org_present", ["subject","O"], "has_org"], None)
    is_self_signed = bool(_safe_get(cert_result, ["is_self_signed"], False))
    validity_days = _safe_get(cert_result, ["validity_days", "days_valid"], None)
    issuer = (_safe_get(cert_result, ["issuer", ["issuer","CN"]], "") or "").lower()
    san_count = _safe_get(cert_result, ["san_count"], None)
    wildcard = bool(_safe_get(cert_result, ["wildcard", "is_wildcard"], False))

    risk_cert = 0.0
    if org_present is False:
        risk_cert += 0.20
    if is_self_signed:
        risk_cert += 0.25
    if isinstance(validity_days, (int, float)) and validity_days < 90:
        risk_cert += 0.10
    if "let's encrypt" in issuer or "lets encrypt" in issuer:
        risk_cert += 0.05
    if isinstance(san_count, int) and san_count <= 1:
        risk_cert += 0.05
    if wildcard:
        risk_cert += 0.05

    tld = (_safe_get(domain_result, ["tld"], "") or "").lower().lstrip(".")
    dlen = _safe_get(domain_result, ["length", "domain_length"], None)
    entropy = float(_safe_get(domain_result, ["shannon_entropy","entropy"], 0.0) or 0.0)

    suspicious_tlds = {"buzz","top","xyz","icu","cn","click","work","shop","online","monster","zip","mov","country"}
    risk_domain = 0.0
    if tld in suspicious_tlds:
        risk_domain += 0.10
    if isinstance(dlen, int) and dlen <= 8:
        risk_domain += 0.05
    if entropy and entropy >= 3.6:
        risk_domain += 0.10

    score = 0.35 * risk_from_ml + 0.30 * min(1.0, risk_brand) + 0.20 * min(1.0, risk_cert) + 0.15 * min(1.0, risk_domain)
    score = max(0.0, min(1.0, score))

    is_phishing = bool(score >= 0.50)
    confidence = float(score)

    reasons = []
    if brand_match:
        reasons.append(f"ブランド偽装の疑い（{brand_name or 'unknown'}）")
    if typo_score and typo_score >= 0.5:
        reasons.append(f"タイポスクワッティング指標が高い({typo_score:.2f})")
    if org_present is False:
        reasons.append("証明書の組織情報が欠如")
    if is_self_signed:
        reasons.append("自己署名証明書")
    if isinstance(validity_days, (int, float)) and validity_days < 90:
        reasons.append(f"短期証明書({int(validity_days)}日)")
    if "let's encrypt" in issuer or "lets encrypt" in issuer:
        reasons.append("無料CA(LE)の使用")
    if tld in suspicious_tlds:
        reasons.append(f"疑わしいTLD(.{tld})")
    if isinstance(dlen, int) and dlen <= 8:
        reasons.append(f"短いドメイン長({dlen})")
    if entropy and entropy >= 3.6:
        reasons.append(f"高エントロピー({entropy:.2f})")
    if not reasons:
        reasons.append("複合リスクの総合判断")

    # Compat aliases for legacy tests and downstreams
    if score >= 0.75:
        risk_level = "high"
    elif score >= 0.50:
        risk_level = "medium"
    else:
        risk_level = "low"
    final_label = "phishing" if is_phishing else "benign"

    out = {
        "is_phishing": is_phishing,
        "confidence": confidence,
        "risk_factors": {
            "ml_low_confidence": risk_from_ml,
            "brand": {
                "match": brand_match,
                "name": brand_name,
                "typo_score": float(typo_score),
                "risk_component": min(1.0, risk_brand),
            },
            "certificate": {
                "org_present": bool(org_present) if org_present is not None else None,
                "is_self_signed": is_self_signed,
                "validity_days": int(validity_days) if isinstance(validity_days, (int, float)) else None,
                "issuer": issuer,
                "san_count": int(san_count) if isinstance(san_count, int) else None,
                "wildcard": wildcard,
                "risk_component": min(1.0, risk_cert),
            },
            "domain": {
                "tld": tld,
                "length": int(dlen) if isinstance(dlen, int) else None,
                "entropy": float(entropy) if entropy else None,
                "risk_component": min(1.0, risk_domain),
            },
            "combined_score": score,
        },
        "reasoning": " / ".join(reasons),

        # ---- Backward-compat keys (non-breaking) ----
        "risk_score": score,      # numeric synonym of confidence
        "risk_level": risk_level, # "low" / "medium" / "high"
        "final_label": final_label,  # "phishing" / "benign"
        "risk": score,            # additional synonym used by some test cells
    }
    return out

In [ ]:
# === Cell 06-B-RISK-TEST ===
dummy_brand = {"is_impersonation": True, "confidence": 0.8, "brand": "PayPal"}
dummy_cert = {"valid_https": True, "issuer": "Let's Encrypt"}
dummy_domain = {"suspicious_tld": True, "levenshtein_to_brand": 2}
out = contextual_risk_assessment(
    domain="paypal-secure-login.info",
    ml_probability=0.34,
    brand_result=dummy_brand,
    cert_result=dummy_cert,
    domain_result=dummy_domain,
)
print("[TEST] contextual_risk_assessment ->", out)
assert isinstance(out, dict), "Output must be dict"
assert "risk" in out or "risk_score" in out or "final_label" in out or "risk_level" in out, "Expect risk-related keys"
print("[TEST] contextual_risk_assessment -> OK")


In [ ]:
# === Cell 06-E-RUN-PARALLEL: run_parallel_evaluation (restore, minimal) ===
import asyncio
from typing import Dict, Any, Tuple, Optional
import pandas as pd

async def run_parallel_evaluation(sample_size: int = 50,
                                  max_concurrent: int = 8,
                                  development_mode: bool = False) -> Tuple[pd.DataFrame, Dict[str, Any]]:
    """Top-level driver that runs parallel evaluation and (optionally) basic analysis.

    Notes:
    - To avoid coupling with 04-5 (which holds analyze_evaluation_results/generate_evaluation_report),
      this function tries to call them *only if they exist in globals()*.
    - If not available, returns (results_df, {}) without analysis (04-5 remains non-modified).
    - Input dataset policy:
        * If development_mode is True, uses a tiny synthetic DataFrame.
        * Else, tries to use a global `samples_df` if present; otherwise returns (empty_df, {}).
    """
    # 1) prepare samples
    if development_mode:
        df = pd.DataFrame({
            "domain": ["paypal-secure-login.info", "example.com", "login-appleid-secure.net"],
            "ml_probability": [0.34, 0.80, 0.12],
        })
    else:
        df = globals().get("samples_df")
        if df is None or not hasattr(df, "head"):
            print("[run_parallel_evaluation] No global samples_df found; returning empty.")
            return pd.DataFrame(), {}

    if sample_size and len(df) > sample_size:
        df = df.head(sample_size).copy()

    # 2) run parallel evaluation
    try:
        results_df = await evaluate_samples_parallel(df, max_concurrent=max_concurrent, show_progress=True)
    except Exception as e:
        print(f"[run_parallel_evaluation] evaluate_samples_parallel failed: {e}")
        return pd.DataFrame(), {}

    if results_df is None or results_df.empty:
        print("[run_parallel_evaluation] No results; returning.")
        return pd.DataFrame(), {}

    # 3) optional analysis (only if 04-5 functions are already in this kernel)
    analysis = {}
    if "analyze_evaluation_results" in globals() and "generate_evaluation_report" in globals():
        try:
            analysis = analyze_evaluation_results(results_df)
            report = generate_evaluation_report(analysis, results_df)
            print(report)
        except Exception as e:
            print(f"[run_parallel_evaluation] analysis/report generation failed: {e}")
            analysis = {}
    else:
        print("[run_parallel_evaluation] 04-5 analysis functions not found; skipped (non-breaking).")

    return results_df, analysis


In [ ]:
# === Cell 06-F-COMPAT: evaluate_async / evaluate_samples_parallel (wrappers) ===
import asyncio
from typing import Dict, Any, Optional
import pandas as pd

# Reuse existing Async API on agent
def _get_eval_agent():
    if 'FixedPhishingDetectionAgent' in globals():
        try:
            return FixedPhishingDetectionAgent()
        except Exception:
            pass
    if 'PhishingDetectionAgent' in globals():
        try:
            return PhishingDetectionAgent()
        except Exception:
            pass
    return None

async def evaluate_async(domain: str,
                         ml_prob: float,
                         semaphore: Optional[asyncio.Semaphore] = None,
                         index: int = 0,
                         agent=None) -> Dict[str, Any]:
    """
    Wrapper for original name compatibility.
    Delegates to agent.analyze_domain_async(domain, ml_prob, index).
    Returns a dict with at least: domain, ml_probability, is_phishing, confidence, reasoning.
    """
    agent = agent or _get_eval_agent()
    if agent is None:
        # graceful fallback: minimal mock
        return {"domain": domain, "ml_probability": ml_prob, "is_phishing": False, "confidence": float(ml_prob or 0.0),
                "reasoning": "[04-4] no agent available (mock)", "risk_level": "low", "risk_score": float(ml_prob or 0.0), "final_label":"benign"}
    async def _run():
        if hasattr(agent, "analyze_domain_async"):
            res = await agent.analyze_domain_async(domain, ml_prob, index)
        else:
            # fallback to evaluate_domain* (sync wrapped)
            if hasattr(agent, "evaluate_domain_fixed"):
                res = await asyncio.to_thread(agent.evaluate_domain_fixed, domain=domain, ml_probability=ml_prob, index=index)
            elif hasattr(agent, "evaluate_domain"):
                res = await asyncio.to_thread(agent.evaluate_domain, domain=domain, ml_probability=ml_prob, index=index)
            else:
                res = {"is_phishing": False, "confidence": 0.0, "reasoning": "no evaluator"}
        if isinstance(res, dict):
            # ---- Backward-compat + hardening (None-safe) ----
            res.setdefault("domain", domain)
            res.setdefault("ml_probability", ml_prob)
            # normalize confidence
            conf = res.get("confidence")
            if conf is None:
                conf = 0.0
            try:
                conf = float(conf)
            except Exception:
                conf = 0.0
            res["confidence"] = conf
            # normalize is_phishing
            if res.get("is_phishing") is None:
                res["is_phishing"] = bool(conf >= 0.5)
            # aliases
            res.setdefault("risk_score", conf)
            if "risk_level" not in res:
                res["risk_level"] = "high" if conf >= 0.75 else ("medium" if conf >= 0.5 else "low")
            if "final_label" not in res:
                res["final_label"] = "phishing" if res.get("is_phishing") else "benign"
            res.setdefault("reasoning", "")
            return res
        # If class-like result, try to serialize
        try:
            d = getattr(res, "__dict__", {}) or {}
            d.setdefault("domain", domain)
            d.setdefault("ml_probability", ml_prob)
            # also ensure minimal keys
            d.setdefault("confidence", 0.0)
            d.setdefault("is_phishing", False)
            d.setdefault("risk_score", d.get("confidence", 0.0))
            d.setdefault("risk_level", "low")
            d.setdefault("final_label", "benign")
            d.setdefault("reasoning", "")
            return d
        except Exception:
            return {"domain": domain, "ml_probability": ml_prob, "is_phishing": False, "confidence": 0.0, "reasoning": "serialize-fallback",
                    "risk_level": "low", "risk_score": 0.0, "final_label": "benign"}
    if semaphore is None:
        return await _run()
    async with semaphore:
        return await _run()

async def evaluate_samples_parallel(samples_df: pd.DataFrame,
                                    max_concurrent: int = 8,
                                    show_progress: bool = True) -> pd.DataFrame:
    """
    Run evaluate_async over rows of samples_df in parallel.
    Expected columns: ['domain', 'ml_probability'] (extra columns are preserved via merge).
    """
    if samples_df is None or samples_df.empty:
        return pd.DataFrame()
    df = samples_df.copy()
    if "domain" not in df.columns or "ml_probability" not in df.columns:
        # try common fallbacks
        domain_col = "domain" if "domain" in df.columns else next((c for c in df.columns if "domain" in c.lower()), None)
        prob_col = "ml_probability" if "ml_probability" in df.columns else next((c for c in df.columns if "prob" in c.lower()), None)
        if not domain_col or not prob_col:
            raise ValueError("[04-4] evaluate_samples_parallel: input df must have domain/ml_probability")
        df = df.rename(columns={domain_col: "domain", prob_col: "ml_probability"})
    sem = asyncio.Semaphore(max_concurrent) if max_concurrent and max_concurrent > 0 else None
    agent = _get_eval_agent()
    tasks = []
    for i, row in df.reset_index(drop=True).iterrows():
        tasks.append(evaluate_async(row["domain"], float(row["ml_probability"]) if row["ml_probability"] is not None else 0.0, sem, i, agent))
    results = []
    # gather with progress print (optional)
    for fut in asyncio.as_completed(tasks):
        res = await fut
        results.append(res)
        if show_progress and len(results) % 10 == 0:
            print(f"[04-4] progress: {len(results)}/{len(tasks)}")
    out_df = pd.DataFrame(results)
    # ensure minimal columns
    for col in ["domain","ml_probability","is_phishing","confidence","reasoning","risk_level","risk_score","final_label"]:
        if col not in out_df.columns:
            out_df[col] = None
    return out_df

In [ ]:
# === Cell 06-E-RUN-PARALLEL-TEST (Jupyter-safe) ===
import pandas as pd
try:
    # Dev-mode run (no dependency on external dataset or 04-5)
    results_df, analysis = await run_parallel_evaluation(sample_size=3, max_concurrent=3, development_mode=True)
    print("[TEST] run_parallel_evaluation -> results shape:", results_df.shape)
    print(results_df.head())
    assert isinstance(results_df, pd.DataFrame), "results_df must be DataFrame"
    assert not results_df.empty, "results_df should not be empty in dev mode"
    assert isinstance(analysis, dict), "analysis must be dict"
    print("[TEST] run_parallel_evaluation -> OK (dev mode)")
except Exception as e:
    print("[TEST] run_parallel_evaluation -> FAILED:", e)
